In [2]:
%conda install openjdk -y

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyspark==3.2.0


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)


3.2.0


In [5]:
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import time
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import datetime as DT
from datetime import date, timedelta as td

import pickle

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

# 1. Functions

In [6]:
# 1. CP data
def read_cp_data(path):
    
    
    df_cp = spark.read.parquet(path)
    
    df_cp = df_cp.withColumn("PRODUCT_PART_NUMBER_IN_CP", F.col("PRODUCT_PART_NUMBER")) \
            .withColumn("ORDER_PLACED_DTTM_CP", F.col("ORDER_PLACED_DTTM"))\
			.withColumn("ORDER_PLACED_YEAR_CP", F.year(F.col("ORDER_PLACED_DTTM"))) \
			.withColumn("ORDER_PLACED_MONTH_CP", F.month(F.col("ORDER_PLACED_DTTM")))
    
    df_cp = df_cp.select("CUSTOMER_ID", "ORDER_ID", "ORDER_LINE_ID",
                               "PRODUCT_PART_NUMBER_IN_CP",
                               "ORDER_PLACED_DTTM_CP",
                               "ORDER_PLACED_YEAR_CP",
                               "ORDER_PLACED_MONTH_CP",
                               "LOCATION_CODE",
                               "ORDER_LINE_QUANTITY",
                               "ORDER_STATUS",
                               "IS_AUTO_SHIP",
                               "IB_COST","OB_COST", "COGS", "AVERAGE_PRICE", "NET_MARGIN")
    
    return df_cp

In [7]:
# 2. Complete clickstream data
def read_clickstream_data(path):
    
    df_clickstream_data = spark.read.parquet(path)

    df_clickstream_data = df_clickstream_data.withColumn("SESSION_YEAR",F.year("SESSION_EST_TIMESTAMP"))\
                               .withColumn("SESSION_MONTH",F.month("SESSION_EST_TIMESTAMP"))
    
    
    
    return df_clickstream_data

In [8]:
# 3. Complete scheduled as order data
def read_as_orders_data(path):
    
    df_as_orders_data = spark.read.parquet(path)

    df_as_orders_data = df_as_orders_data.withColumn("ORDER_PLACED_DATE",F.to_date("ORDER_PLACED_DTTM"))\
                                          .withColumn("ORDER_PLACED_YEAR",F.year("ORDER_PLACED_DTTM"))\
                                          .withColumn("ORDER_PLACED_MONTH",F.month("ORDER_PLACED_DTTM"))
    
    return df_as_orders_data

In [9]:
# 4. SKU-Month pool
def read_sku_month_candidate_data(path):
    
    pd_sku_month = pd.read_csv(path)
    
    return pd_sku_month


In [10]:
# 5. Clickstream slice
def clickstream_slice_pandas(df_data, sample_sku, sample_year, sample_month):
    
    df_slice = df_data.where((F.col("PRODUCT_PART_NUMBER")== str(sample_sku))\
                                            & (F.col("SESSION_YEAR") == sample_year) \
                                            & (F.col("SESSION_MONTH") == sample_month))
    
    pd_slice = df_slice.toPandas()
    
    return df_slice, pd_slice



In [11]:
# 6. Clickstream count summaries

def clickstream_counts_by_instock_status(pd_clickstream_data_slice):
    
    clickstream_identifier_variables = ["CUSTOMER_ID",
                              "SESSION_DATE",
                              "SESSION_EST_TIMESTAMP",
                              "PRODUCT_PART_NUMBER",
                              "STATUS",
                              "DEVICE_CATEGORY"]

    is_clickstream_counts = len(pd_clickstream_data_slice[pd_clickstream_data_slice["STATUS"] == "IN STOCK"][clickstream_identifier_variables].drop_duplicates())
    oos_clickstream_counts = len(pd_clickstream_data_slice[pd_clickstream_data_slice["STATUS"] == "OUT OF STOCK"][clickstream_identifier_variables].drop_duplicates())
    total_clickstream_counts = len(pd_clickstream_data_slice[clickstream_identifier_variables].drop_duplicates())
    
    return is_clickstream_counts, oos_clickstream_counts, total_clickstream_counts

In [12]:
# 7. Units sold summaries
def units_sold_by_as_status(df_cp_sku_month_data):
    
    non_as_units_sold = df_cp_sku_month_data.filter(F.col("IS_AUTO_SHIP") == "NON AUTO SHIP").select(F.sum("ORDER_LINE_QUANTITY")).collect()[0][0]
    as_units_sold = df_cp_sku_month_data.filter(F.col("IS_AUTO_SHIP").isin(["AUTO SHIP", "AUTO SHIP NOW"])).select(F.sum("ORDER_LINE_QUANTITY")).collect()[0][0]
    total_units_sold = df_cp_sku_month_data.select(F.sum("ORDER_LINE_QUANTITY")).collect()[0][0]

    
    return non_as_units_sold, as_units_sold, total_units_sold

In [13]:
# 8. Find clickstream cutoff dates

def find_clickstream_cutoff_dates(pd_clickstream_data_slice):
    

    
    
    # 1. Dummy variables that indicate IS and OOS
    
    pd_clickstream_data_slice["IS"] = np.where(pd_clickstream_data_slice["STATUS"] == "IN STOCK", 1, 0)
    pd_clickstream_data_slice["OOS"] = np.where(pd_clickstream_data_slice["STATUS"] == "OUT OF STOCK", 1, 0)
    
    pd_clickstream_is_oos_counts_by_date = pd_clickstream_data_slice.groupby(["SESSION_DATE"])[["IS", "OOS"]].sum().reset_index()
    pd_clickstream_is_oos_counts_by_date["IS_ADJ"] = np.where(pd_clickstream_is_oos_counts_by_date["IS"]/(pd_clickstream_is_oos_counts_by_date["IS"]+pd_clickstream_is_oos_counts_by_date["OOS"])<0.1, 0, pd_clickstream_is_oos_counts_by_date["IS"]) 
    pd_clickstream_is_oos_counts_by_date["OOS_ADJ"] = np.where(pd_clickstream_is_oos_counts_by_date["OOS"]/(pd_clickstream_is_oos_counts_by_date["IS"]+pd_clickstream_is_oos_counts_by_date["OOS"])<0.1, 0, pd_clickstream_is_oos_counts_by_date["OOS"]) 
    
    # 2. Generate status switching variable

    N = len(pd_clickstream_is_oos_counts_by_date)

    for row in range(N):

        if row == 0:

            temp = pd_clickstream_is_oos_counts_by_date.iloc[row:(row+1),].reset_index()
            is_counts = temp["IS_ADJ"][0]
            oos_counts = temp["OOS_ADJ"][0]
            
            # initialization for the first date in this monthly sample 
            if (is_counts == 0) & (oos_counts == 0):
                temp_status = "IS"
            elif (is_counts > 0) & (oos_counts == 0):
                temp_status = "IS"
            elif (is_counts == 0) & (oos_counts > 0):
                temp_status = "OOS"
            elif (is_counts > 0) & (oos_counts > 0):
                temp_status = "SWITCHING TO IS"

            temp["SWITCHING_STATUS"] = temp_status

            pd_clickstream_cutoff_point_detection = temp

        else:

            temp_t_minus_1 = pd_clickstream_cutoff_point_detection.iloc[(row-1):row,].reset_index()
            switching_status_t_minus_1 = temp_t_minus_1["SWITCHING_STATUS"][0]

            temp_t = pd_clickstream_is_oos_counts_by_date.iloc[row:(row+1),].reset_index()
            is_counts = temp_t["IS_ADJ"][0]
            oos_counts = temp_t["OOS_ADJ"][0]
            
            if (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "SWITCHING TO IS"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"

            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "SWITCHING TO IS"    
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS"

            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "OOS"    
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "OOS"

            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "IS"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "OOS"    
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"

            else:
                switching_status_t = np.nan
                print("There is NULL value in the table.")

            temp_t["SWITCHING_STATUS"] = switching_status_t
            
            pd_clickstream_cutoff_point_detection = pd_clickstream_cutoff_point_detection.append(temp_t)


    pd_clickstream_cutoff_points = pd_clickstream_cutoff_point_detection[pd_clickstream_cutoff_point_detection["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"])][["SESSION_DATE","SWITCHING_STATUS"]]
    
    
    return pd_clickstream_cutoff_point_detection, pd_clickstream_cutoff_points


In [14]:
# 9. Clickstream RDD customer list 
def find_clickstream_rdd_relevant_customers(pd_clickstream_cutoff_points, pd_clickstream_cutoff_point_detection, pd_clickstream_data_slice, sufficiency_ratio, initial_window_length, first_layer_index):
    
    
    # print("==========CONSTRUCTING CLICKSTREAM RDD CUSTOMER SAMPLE FOR " + first_layer_index + " ==========")
    
    # clickstream_rdd_customers_temp = pd.DataFrame(columns = ["CUSTOMER_ID"])
    
    clickstream_rdd_customers_temp = []

    # 1. RDD sample generation
    for k in range(len(pd_clickstream_cutoff_points)):
        
        window_length = initial_window_length

        cutoff_point = pd_clickstream_cutoff_points.iloc[k,]["SESSION_DATE"]

        cutoff_point_name = pd_clickstream_cutoff_points.iloc[k,]["SWITCHING_STATUS"]

        # Check T days prior to the cutoff date and T days after the cutoff date
        while window_length > 0:

            # Window start date
            window_left_end = max(cutoff_point - DT.timedelta(days=window_length), min(pd_clickstream_cutoff_point_detection["SESSION_DATE"]))
            # Window end date
            window_right_end = min(cutoff_point + DT.timedelta(days=(window_length+1)), max(pd_clickstream_cutoff_point_detection["SESSION_DATE"])+ DT.timedelta(days=1))

            pd_cutoff_point_detection_rdd_sample_temp = pd_clickstream_cutoff_point_detection[(pd_clickstream_cutoff_point_detection["SESSION_DATE"] >= window_left_end) & (pd_clickstream_cutoff_point_detection["SESSION_DATE"] < window_right_end) ]

            # [1] We want to make sure this sample only has the cutoff point that is under inspection
            if np.sum(pd_cutoff_point_detection_rdd_sample_temp["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"]))>1:
                window_length = window_length - 1
                if window_length == 0:
                    temp=0
                #   print("THIS IS THE SHORTEST RDD SAMPLE OF " + str(cutoff_point) + " YET IT IS STILL NOT CLEAN")
                continue
            else:
                temp=0
                # print("FOUND A CLEAN RDD SAMPLE FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("CHECKING DATA SUFFICIENCY")

            # [1] Clickstream RDD horizon
            clickstream_rdd_sample_temp = pd_clickstream_data_slice[ (pd_clickstream_data_slice["SESSION_DATE"]>=window_left_end) & (pd_clickstream_data_slice["SESSION_DATE"]< window_right_end)]

            # [2] Remove duplicate clickstream data points   
            clickstream_identifier_variables = ["CUSTOMER_ID", "SESSION_EST_TIMESTAMP",
                            "PRODUCT_PART_NUMBER", 
                            "STATUS", 
                            "DEVICE_CATEGORY",
                            "SESSION_DATE", "SESSION_YEAR", "SESSION_MONTH"]
            clickstream_rdd_sample_cleaned = clickstream_rdd_sample_temp[clickstream_identifier_variables].drop_duplicates()

            # [3] Ensure at least 50% of days on both sides of the cutoff point need to have data 
            # Summarize daily data points
            clickstream_counts_distribution_across_days = clickstream_rdd_sample_cleaned.groupby(["SESSION_DATE"])["CUSTOMER_ID"].count().reset_index()

            if (clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] < cutoff_point]["SESSION_DATE"].nunique() >= window_length*sufficiency_ratio) & \
             (clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] > cutoff_point]["SESSION_DATE"].nunique() >= window_length*sufficiency_ratio):
                # print("WE FOUND SUFFICIENT DATA FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("LEFT SIDE HAS " + str(clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] < cutoff_point]["SESSION_DATE"].nunique()) + " DAYS OF DATA")
                # print("RIGHT SIDE HAS " + str(clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] > cutoff_point]["SESSION_DATE"].nunique()) + " DAYS OF DATA")
                # print("THIS RDD SAMPLE IS COLLECTED FOR REGRESSION AND ANALYSES")
                temp = 0
                break

            else:
                window_length = window_length - 1
                if window_length == 0:   
                    # print("THIS IS THE SHORTEST RDD SAMPLE OF " + str(cutoff_point) + " YET IT DOES NOT HAVE SUFFICIENT DATA")
                    temp = 0
                continue

        if window_length == 0:
            # print("UNFORTUNATELY, WE CANNOT FIND AN IDEAL RDD SAMPLE FOR CUTOFF POINT: " + str(cutoff_point))
            temp = 0
            continue

        else:
            # print("WE HAVE FOUND THE RDD SAMPLE FOR REGRESSION FOR CUTOFF POINT: " + str(cutoff_point))
            # print("THE LEFT END DATE IS: " + str(window_left_end))
            # print("THE RIGHT END DATE IS: " + str(window_right_end))
            temp = 0
            
            # [1] Collect the customers

            # clickstream_rdd_customers_temp0 = clickstream_rdd_sample_cleaned[["CUSTOMER_ID"]].drop_duplicates().reset_index()[["CUSTOMER_ID"]]
            
            clickstream_rdd_customers_temp0 = clickstream_rdd_sample_cleaned[["CUSTOMER_ID"]].drop_duplicates()["CUSTOMER_ID"].to_list()
            
            second_layer_index = str(cutoff_point) + "_" + str(cutoff_point_name) + "_" + str(window_length)

            # print("(CLICKSTREAM) THE NUMBER OF CUSTOMERS IN " + first_layer_index + " SAMPLE " + second_layer_index + " IS " + str(len(clickstream_rdd_customers_temp0)))

            # clickstream_rdd_customers_temp = clickstream_rdd_customers_temp.append(clickstream_rdd_customers_temp0).drop_duplicates().reset_index()[["CUSTOMER_ID"]]
            clickstream_rdd_customers_temp = clickstream_rdd_customers_temp + clickstream_rdd_customers_temp0
            clickstream_rdd_customers_temp = [*set(clickstream_rdd_customers_temp)]
    
    # print("(CLICKSTREAM) THE NUMBER OF CUSTOMERS IN " + first_layer_index + " IS " + str(len(clickstream_rdd_customers_temp)))

    return clickstream_rdd_customers_temp

In [15]:
# 10. Clickstream RDD sample construction
def construct_clickstream_rdd_samples(pd_clickstream_cutoff_points, pd_clickstream_cutoff_point_detection, pd_clickstream_data_slice, sufficiency_ratio, initial_window_length, first_layer_index, df_cp):
    
    
    print("==========CONSTRUCTING CLICKSTREAM RDD SAMPLE FOR " + first_layer_index + " ==========")
    
    
    start_time_00 = time.time()
    
    clickstream_rdd_sample_collection_temp = {}
    
    # 1. RDD sample generation
    for k in range(len(pd_clickstream_cutoff_points)):
        
        window_length = initial_window_length

        cutoff_point = pd_clickstream_cutoff_points.iloc[k,]["SESSION_DATE"]

        cutoff_point_name = pd_clickstream_cutoff_points.iloc[k,]["SWITCHING_STATUS"]

        # Check T days prior to the cutoff date and T days after the cutoff date
        while window_length > 0:

            # Window start date
            window_left_end = max(cutoff_point - DT.timedelta(days=window_length), min(pd_clickstream_cutoff_point_detection["SESSION_DATE"]))
            # Window end date
            window_right_end = min(cutoff_point + DT.timedelta(days=(window_length+1)), max(pd_clickstream_cutoff_point_detection["SESSION_DATE"])+ DT.timedelta(days=1))

            pd_cutoff_point_detection_rdd_sample_temp = pd_clickstream_cutoff_point_detection[(pd_clickstream_cutoff_point_detection["SESSION_DATE"] >= window_left_end) & (pd_clickstream_cutoff_point_detection["SESSION_DATE"] < window_right_end) ]

            # [1] We want to make sure this sample only has the cutoff point that is under inspection
            if np.sum(pd_cutoff_point_detection_rdd_sample_temp["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"]))>1:
                window_length = window_length - 1
                if window_length == 0:
                    # print("THIS IS THE SHORTEST RDD SAMPLE OF " + str(cutoff_point) + " YET IT IS STILL NOT CLEAN")
                    temp = 1
                continue 
            else:
                # print("FOUND A CLEAN RDD SAMPLE FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("CHECKING DATA SUFFICIENCY")
                temp = 1

            # [1] Clickstream RDD horizon
            clickstream_rdd_sample_temp = pd_clickstream_data_slice[ (pd_clickstream_data_slice["SESSION_DATE"]>=window_left_end) & (pd_clickstream_data_slice["SESSION_DATE"]< window_right_end)]

            # [2] Remove duplicate clickstream data points   
            clickstream_identifier_variables = ["CUSTOMER_ID", "SESSION_EST_TIMESTAMP",
                            "PRODUCT_PART_NUMBER", 
                            "STATUS", 
                            "DEVICE_CATEGORY",
                            "SESSION_DATE", "SESSION_YEAR", "SESSION_MONTH"]
            clickstream_rdd_sample_cleaned = clickstream_rdd_sample_temp[clickstream_identifier_variables].drop_duplicates()

            # [3] Ensure at least 50% of days on both sides of the cutoff point need to have data 
            # Summarize daily data points
            clickstream_counts_distribution_across_days = clickstream_rdd_sample_cleaned.groupby(["SESSION_DATE"])["CUSTOMER_ID"].count().reset_index()

            if (clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] < cutoff_point]["SESSION_DATE"].nunique() >= window_length*sufficiency_ratio) & \
             (clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] > cutoff_point]["SESSION_DATE"].nunique() >= window_length*sufficiency_ratio):
                # print("WE FOUND SUFFICIENT DATA FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("LEFT SIDE HAS " + str(clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] < cutoff_point]["SESSION_DATE"].nunique()) + " DAYS OF DATA")
                # print("RIGHT SIDE HAS " + str(clickstream_counts_distribution_across_days[clickstream_counts_distribution_across_days["SESSION_DATE"] > cutoff_point]["SESSION_DATE"].nunique()) + " DAYS OF DATA")
                # print("THIS RDD SAMPLE IS COLLECTED FOR REGRESSION AND ANALYSES")
                temp = 1
                break

            else:
                window_length = window_length - 1
                if window_length == 0:   
                    # print("THIS IS THE SHORTEST RDD SAMPLE OF " + str(cutoff_point) + " YET IT DOES NOT HAVE SUFFICIENT DATA")
                    temp = 1
                continue

        if window_length == 0:
            # print("UNFORTUNATELY, WE CANNOT FIND AN IDEAL RDD SAMPLE FOR CUTOFF POINT: " + str(cutoff_point))
            temp = 1
            continue

        else:
            # print("WE HAVE FOUND THE RDD SAMPLE FOR REGRESSION FOR CUTOFF POINT: " + str(cutoff_point))
            # print("THE LEFT END DATE IS: " + str(window_left_end))
            # print("THE RIGHT END DATE IS: " + str(window_right_end))
            temp = 1

            # 2. Feature Engineering

            # [1] The precise timestamp of the cutoff point
            
            # We make the following update: if the status switches from OOS to IS then we use the first IS timestamp within the cutoff date as the cutoff timestamp. Similar for IS -> OOS.  
            
            clickstream_rdd_sample_on_cutoff_date = clickstream_rdd_sample_cleaned[clickstream_rdd_sample_cleaned["SESSION_DATE"] == cutoff_point]
            
            clickstream_rdd_sample_on_cutoff_date["STATUS_RANK"] = clickstream_rdd_sample_on_cutoff_date.groupby("STATUS")["SESSION_EST_TIMESTAMP"].rank(method = "dense", ascending = True).astype(int)

            if cutoff_point_name == "SWITCHING TO IS":

                cutoff_point_timestamp = clickstream_rdd_sample_on_cutoff_date[clickstream_rdd_sample_on_cutoff_date["STATUS_RANK"]==1][clickstream_rdd_sample_on_cutoff_date["STATUS"] == "IN STOCK"].reset_index().iloc[0]["SESSION_EST_TIMESTAMP"]

            else:

                cutoff_point_timestamp = clickstream_rdd_sample_on_cutoff_date[clickstream_rdd_sample_on_cutoff_date["STATUS_RANK"]==1][clickstream_rdd_sample_on_cutoff_date["STATUS"] == "OUT OF STOCK"].reset_index().iloc[0]["SESSION_EST_TIMESTAMP"]

                
            # [2] Treatment variable    
            
            clickstream_rdd_sample_cleaned["TREATMENT_VARIABLE"] = np.where(clickstream_rdd_sample_cleaned["STATUS"] == "IN STOCK", 1, 0)


            # [3] Generate the hour difference between the timestamp and the cutoff point
            
            # We make the following update: We are going to drop data points that are "not consistent with RDD design"
           
            clickstream_rdd_sample_cleaned["RUNNING_VARIABLE"] = (clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP"] - cutoff_point_timestamp).dt.total_seconds()/3600

            if cutoff_point_name == "SWITCHING TO IS":

                clickstream_rdd_sample_cleaned["DROP_INDICATOR"] = np.where( ((clickstream_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 1) & (clickstream_rdd_sample_cleaned["RUNNING_VARIABLE"] < 0)) \
                                                                              | ((clickstream_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 0) & (clickstream_rdd_sample_cleaned["RUNNING_VARIABLE"] >= 0)), 1, 0)
            
                # print("BEFORE DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(clickstream_rdd_sample_cleaned)))
                
                clickstream_rdd_sample_cleaned = clickstream_rdd_sample_cleaned[clickstream_rdd_sample_cleaned["DROP_INDICATOR"]==0]
                
                # print("AFTER DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(clickstream_rdd_sample_cleaned)))
                
            else:
                
                clickstream_rdd_sample_cleaned["DROP_INDICATOR"] = np.where( ((clickstream_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 1) & (clickstream_rdd_sample_cleaned["RUNNING_VARIABLE"] >= 0)) \
                                                                              | ((clickstream_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 0) & (clickstream_rdd_sample_cleaned["RUNNING_VARIABLE"] < 0)), 1, 0)

                # print("BEFORE DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(clickstream_rdd_sample_cleaned)))
                
                clickstream_rdd_sample_cleaned = clickstream_rdd_sample_cleaned[clickstream_rdd_sample_cleaned["DROP_INDICATOR"]==0]
                
                # print("AFTER DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(clickstream_rdd_sample_cleaned)))
            
            # [4] Outcome Variable Construction

            customer_id_list_in_clickstream_rdd_sample = clickstream_rdd_sample_cleaned[["CUSTOMER_ID"]].drop_duplicates()["CUSTOMER_ID"].to_list()

            # Add filtering variables
            clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP_UB"] = clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP"] + pd.Timedelta(hours=24)
            clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP_END"] = clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP"] + pd.Timedelta(hours=24*365)

            # Max timestamp possible for CP calculation
            max_timestamp = np.max(clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP_END"])

            # Min timestamp possible for CP calculation
            min_timestamp = np.min(clickstream_rdd_sample_cleaned["SESSION_EST_TIMESTAMP"])

            # Focus on customers in clickstream data & Focus on time period that is relevant
            df_cp_clickstream = df_cp.filter( F.col("CUSTOMER_ID").isin(customer_id_list_in_clickstream_rdd_sample)) 
                                             
                                             # & \
                                             # (F.col("ORDER_PLACED_DTTM") >= min_timestamp) & \
                                             # (F.col("ORDER_PLACED_DTTM") <= max_timestamp) )

            # df_cp_clickstream = df_cp_clickstream.withColumn("NET_MARGIN_V2",  F.col("AVERAGE_PRICE")-F.col("IB_COST")-F.col("OB_COST")-F.col("COGS"))

            # Transform the small cp data into pandas dataframe

            print("==========Converting CP data into Pandas data frame==========")
            start_time = time.time()
            pd_cp_clickstream = df_cp_clickstream.toPandas()
            pd_cp_clickstream["NET_MARGIN_V2"] = pd_cp_clickstream["AVERAGE_PRICE"] - pd_cp_clickstream["IB_COST"] - pd_cp_clickstream["OB_COST"] - pd_cp_clickstream["COGS"]
            pd_cp_clickstream["ORDER_LINE_QUANTITY"] = pd_cp_clickstream["ORDER_LINE_QUANTITY"].astype(float)
            print("--- %s seconds ---" % (time.time() - start_time))

            # Columns we need from clickstream data
            clickstream_relevant_columns = ["CUSTOMER_ID", "SESSION_EST_TIMESTAMP", "SESSION_EST_TIMESTAMP_UB", "SESSION_EST_TIMESTAMP_END",
                            "PRODUCT_PART_NUMBER", 
                            "STATUS", 
                            "DEVICE_CATEGORY", 
                            "SESSION_YEAR", "SESSION_MONTH", 
                            "RUNNING_VARIABLE", "TREATMENT_VARIABLE"]
            clickstream_rdd_sample_cleaned_adj = clickstream_rdd_sample_cleaned[clickstream_relevant_columns]

            start_time_0 = time.time()

            # alternative way
            for row in range(len(clickstream_rdd_sample_cleaned_adj)):

                # print("==========NOW GENERATING CP FOR DATA POINT: " + str(row) + " ==========")

                # start_time = time.time()

                # Loop through each clickstream data point

                clickstream_dp = clickstream_rdd_sample_cleaned_adj.iloc[row:(row+1),:].reset_index()

                # info needed to be added to CP data
                customer_id = clickstream_dp["CUSTOMER_ID"][0]
                product_part_number = clickstream_dp["PRODUCT_PART_NUMBER"][0]
                timestamp_lb = clickstream_dp["SESSION_EST_TIMESTAMP"][0]
                timestamp_ub = clickstream_dp["SESSION_EST_TIMESTAMP_UB"][0]
                timestamp_end = clickstream_dp["SESSION_EST_TIMESTAMP_END"][0]
                device_type = clickstream_dp["DEVICE_CATEGORY"][0]
                instock_status = clickstream_dp["STATUS"][0]
                running_variable = clickstream_dp["RUNNING_VARIABLE"][0]
                treatment_variable = clickstream_dp["TREATMENT_VARIABLE"][0]

                # get CP data that corresponds to the customer id of the clickstream dp
                pd_cp_clickstream_dp = pd_cp_clickstream[pd_cp_clickstream["CUSTOMER_ID"] == customer_id]

                # add the info from the clickstream dp into CP data
                pd_cp_clickstream_dp["PRODUCT_PART_NUMBER"] = product_part_number
                pd_cp_clickstream_dp["SESSION_EST_TIMESTAMP"] = timestamp_lb
                pd_cp_clickstream_dp["SESSION_EST_TIMESTAMP_UB"] = timestamp_ub
                pd_cp_clickstream_dp["SESSION_EST_TIMESTAMP_END"] = timestamp_end
                pd_cp_clickstream_dp["DEVICE_CATEGORY"] = device_type
                pd_cp_clickstream_dp["STATUS"] = instock_status
                pd_cp_clickstream_dp["RUNNING_VARIABLE"] = running_variable
                pd_cp_clickstream_dp["TREATMENT_VARIABLE"] = treatment_variable

                # if len(pd_cp_clickstream_dp) == 0:

                #    print("THIS DATA POINT HAS NO CP RECORD")

                # else:

                #    print("THIS DATA POINT HAS CP RECORD: " + str(len(pd_cp_clickstream_dp)))


                pd_cp_clickstream_dp["CP_INCLUDED"] = np.where(

                    ((pd_cp_clickstream_dp['PRODUCT_PART_NUMBER_IN_CP'] != pd_cp_clickstream_dp['PRODUCT_PART_NUMBER']) & (pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP']) & (pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP_UB']))\
                    | \

                    ((pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP_UB']) & (pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP_END']))

                    , 1, 0)
                
                
                pd_cp_clickstream_dp["CP_INCLUDED_V2"] = np.where(

                (pd_cp_clickstream_dp['PRODUCT_PART_NUMBER_IN_CP'] != pd_cp_clickstream_dp['PRODUCT_PART_NUMBER']) \
                 & ((pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP']) \
                    & (pd_cp_clickstream_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_clickstream_dp['SESSION_EST_TIMESTAMP_END']))

                , 1, 0)

                pd_cp_clickstream_dp["CP"] = pd_cp_clickstream_dp["CP_INCLUDED"] * pd_cp_clickstream_dp["NET_MARGIN"] * pd_cp_clickstream_dp["ORDER_LINE_QUANTITY"]
                pd_cp_clickstream_dp["CP_V2"] = pd_cp_clickstream_dp["CP_INCLUDED"] * pd_cp_clickstream_dp["NET_MARGIN_V2"] * pd_cp_clickstream_dp["ORDER_LINE_QUANTITY"]

                pd_cp_clickstream_dp["CP_V3"] = pd_cp_clickstream_dp["CP_INCLUDED_V2"] * pd_cp_clickstream_dp["NET_MARGIN"] * pd_cp_clickstream_dp["ORDER_LINE_QUANTITY"]
                pd_cp_clickstream_dp["CP_V4"] = pd_cp_clickstream_dp["CP_INCLUDED_V2"] * pd_cp_clickstream_dp["NET_MARGIN_V2"] * pd_cp_clickstream_dp["ORDER_LINE_QUANTITY"]

                
                groupVars = ["PRODUCT_PART_NUMBER", "CUSTOMER_ID", 
                             "SESSION_EST_TIMESTAMP", "SESSION_EST_TIMESTAMP_UB", "SESSION_EST_TIMESTAMP_END", 
                             "STATUS", 
                             "DEVICE_CATEGORY",
                             "RUNNING_VARIABLE", "TREATMENT_VARIABLE"]

                pd_agg_cp_clickstream_dp = pd_cp_clickstream_dp.groupby(groupVars)[["CP","CP_V2","CP_V3","CP_V4"]].sum().reset_index()

                if row == 0:

                    pd_clickstream_cp_outcome =  pd_agg_cp_clickstream_dp

                else:

                    pd_clickstream_cp_outcome = pd_clickstream_cp_outcome.append(pd_agg_cp_clickstream_dp)

                # print("==========GENERATION IS COMPLETED FOR DATA POINT: " + str(row) + " ==========")
                # print("--- %s seconds ---" % (time.time() - start_time))


            # print("--- TOTAL --- %s seconds ---" % (time.time() - start_time_0))


            clickstream_rdd_sample_cleaned["CUSTOMER_ID"] = clickstream_rdd_sample_cleaned["CUSTOMER_ID"].astype(str)
            pd_clickstream_cp_outcome["CUSTOMER_ID"] = pd_clickstream_cp_outcome["CUSTOMER_ID"].astype(str)

            rdd_sample = pd.merge(clickstream_rdd_sample_cleaned, pd_clickstream_cp_outcome, on = groupVars, how = "left")

            second_layer_index = str(cutoff_point) + "_" + str(cutoff_point_name) + "_" + str(window_length)

            print("CP NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP"].isna())/len(rdd_sample)))
            print("CP_V2 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V2"].isna())/len(rdd_sample)))
            print("CP_V3 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V3"].isna())/len(rdd_sample)))
            print("CP_V4 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V4"].isna())/len(rdd_sample)))

            rdd_sample = rdd_sample.fillna({"CP":0,
                                           "CP_V2":0,
                                            "CP_V3":0,
                                            "CP_V4":0})

            clickstream_rdd_sample_collection_temp[second_layer_index] = rdd_sample
        
        
    print("--- TOTAL RUNTIME FOR RDD CONSTRUCTION OF" + first_layer_index + "IS " +  str(time.time() - start_time_00))
        
    return clickstream_rdd_sample_collection_temp

In [16]:
# 11. Add kernel weights to RDD regression

def kernel(R, h, a1, a2):
    indicator = (np.abs(R) <= h).astype(float)
    return (indicator * (1 - np.abs(R)/h) / a1) ** a2


In [17]:
# 12. Clickstream RDD
def clickstream_rdd_modeling(clickstream_rdd_sample_collection, first_layer_index, cp_variable, K, regression_type, h, a1, a2, bootstrapping_method, boostramp_sample_size):

    clickstream_rdd_estimate_collection = {}
    
    print("========== RUNNING CLICKSTREAM RDD FOR " + first_layer_index + " ==========")
    
    start_time_00 = time.time()
    
    if regression_type == "SEPARATE_UNWEIGHTED":

        for sample_index, pd_rdd_sample in clickstream_rdd_sample_collection.items():
            print("========== NOW RUNNING SEPARATE_UNWEIGHTED REGRESSION FOR SAMPLE: " + str(sample_index) + "==========")

            print("DATA POINTS: " + str(len(pd_rdd_sample)))

            print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
            pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
            pd_rdd_sample

            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
            pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
            pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":
                
                plt.figure(0)

                x1 = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE_1"] <= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] <= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] >= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] >= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()
                
                
                plt.figure(1)
            
                # another version of the graph
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show()

            else:

                
                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] <= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] <= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] >= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"] >= 0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()        
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show() 

            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]


                results = sm.OLS(Y, X).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(CLICKSTREAM) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:

                estimate_list = []
                for s in range(boostramp_sample_size):
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)
                    
                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.OLS(Y, X).fit()
                    
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                    
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(CLICKSTREAM) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


            clickstream_rdd_estimate_collection[sample_index] = {}
            clickstream_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
            clickstream_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err


    elif regression_type == "SEPARATE_WEIGHTED":

        for sample_index, pd_rdd_sample in clickstream_rdd_sample_collection.items():
            print("========== NOW RUNNING SEPARATE_WEIGHTED REGRESSION FOR SAMPLE: " + str(sample_index) + "==========")

            print("DATA POINTS: " + str(len(pd_rdd_sample)))

            print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
            pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
            pd_rdd_sample

            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
            pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
            pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":

                
                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show()

            else:

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()      
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show() 

            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]


            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(CLICKSTREAM) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:
                
                estimate_list = []
                
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(CLICKSTREAM) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


            clickstream_rdd_estimate_collection[sample_index] = {}
            clickstream_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
            clickstream_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err
		
    elif regression_type == "JOINT_WEIGHTED":

        index = 1
        
        for sample_index_temp, pd_rdd_sample_temp in clickstream_rdd_sample_collection.items():
            
            print("========== NOW RUNNING JOINT_WEIGHTED REGRESSION ==========")
            print("FOR SAMPLE " + sample_index_temp + ", DATA POINTS: " + str(len(pd_rdd_sample_temp)))

            pd_rdd_sample_temp["SAMPLE_INDEX"] = index

            if index == 1:

                pd_rdd_sample = pd_rdd_sample_temp
                sample_index = sample_index_temp

            else:

                pd_rdd_sample = pd_rdd_sample.append(pd_rdd_sample_temp)
                sample_index = sample_index_temp

            index = index + 1
            
            print("JOINT SAMPLE HAS DATA POINTS: " + str(len(pd_rdd_sample)))


        print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
        pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
        pd_rdd_sample

        print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

        print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
        pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
        pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
        pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
        pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]



        if np.max(pd_rdd_sample["SAMPLE_INDEX"]) == 1:
            
            print("NO NEED TO JOIN AND PROCEED AS SEPARATE REGRESSION")

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========") 
            
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] >= -h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["RUNNING_VARIABLE_1"] <= h) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()

                
                # another version of the graph
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")   
                
                plt.show()
                
            else:

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()        
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")

                plt.show()
                
            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(CLICKSTREAM) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:

                estimate_list = []
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(CLICKSTREAM) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


        else:
            
            index_var_name_list = []
            
            for m in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1):
                
                index_var_name = "SAMPLE_INDEX" + "_" + str(m)
                pd_rdd_sample[index_var_name] = np.where(pd_rdd_sample["SAMPLE_INDEX"] == m, 1, 0)
                index_var_name_list = index_var_name_list + [index_var_name]

            vars_for_interaction_with_sample_index = ["RUNNING_VARIABLE_1", "RUNNING_VARIABLE_2", "RUNNING_VARIABLE_3", "RUNNING_VARIABLE_4",
                                                      "RUNNING_x_TREATMENT_VARIABLE_1", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_2", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_3", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_4"]
            
            interaction_var_name_list = []
            
            for m1 in index_var_name_list:
                for m2 in vars_for_interaction_with_sample_index:
                    
                    interaction_var_name = m2 + "_" + m1
                    pd_rdd_sample[interaction_var_name] = pd_rdd_sample[m1] * pd_rdd_sample[m2]
                    interaction_var_name_list = interaction_var_name_list + [interaction_var_name]


            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] \
            + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)] \
            + ["RUNNING_VARIABLE_" + str(n1)  + "_" + "SAMPLE_INDEX_" + str(n2) for n1 in np.arange(1,K+1) for n2 in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1)] \
            + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n1)  + "_" + "SAMPLE_INDEX_" + str(n2) for n1 in np.arange(1,K+1) for n2 in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(CLICKSTREAM) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:
                
                estimate_list = []
                
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                    
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(CLICKSTREAM) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(CLICKSTREAM) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

                
        clickstream_rdd_estimate_collection[sample_index] = {}
        clickstream_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
        clickstream_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err

    print("--- TOTAL RUNTIME OF CLICKSTREAM RDD FOR" + first_layer_index + "IS " +  str(time.time() - start_time_00))
        
    return clickstream_rdd_estimate_collection




In [18]:
# 13. Clickstream RDD result summaries


def clickstream_rdd_result_summary(clickstream_rdd_estimate_collection, first_layer_index):
    
    print("========== RUNNING CLICKSTREAM RDD FOR " + first_layer_index + " ==========")
    start_time_00 = time.time()
    
    clickstream_rdd_estimate_value_list = []
    clickstream_rdd_estimate_std_error_list = []

    for result_index, result_sample in clickstream_rdd_estimate_collection.items():

        clickstream_rdd_estimate_value_list += [result_sample["Estimate"]]
        clickstream_rdd_estimate_std_error_list += [result_sample["StdError"]]

    if len(clickstream_rdd_estimate_value_list) == 1:
        final_clickstream_estimate_value = clickstream_rdd_estimate_value_list[0]
        final_clickstream_estimate_std_error = clickstream_rdd_estimate_std_error_list[0]
        final_clickstream_z_score = final_clickstream_estimate_value/final_clickstream_estimate_std_error
        final_clickstream_statistical_significance_p05 = np.abs(final_clickstream_z_score) >= 1.96
        final_clickstream_statistical_significance_p10 = np.abs(final_clickstream_z_score) >= 1.645
        print("(CLICKSTREAM) FINAL RDD ESTIMATE FOR " + first_layer_index + " IS: " + str(final_clickstream_estimate_value))
        print("(CLICKSTREAM) FINAL RDD ESTIMATE STD ERROR FOR " + first_layer_index + " IS: "+ str(final_clickstream_estimate_std_error))
        print("(CLICKSTREAM) FINAL Z SCORE FOR " + first_layer_index + " IS: " + str(final_clickstream_z_score))
        print("(CLICKSTREAM) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 5% FOR " + first_layer_index + " IS: " + str(final_clickstream_statistical_significance_p05))
        print("(CLICKSTREAM) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 10% FOR " + first_layer_index + " IS: " + str(final_clickstream_statistical_significance_p10))

    else:

        np_clickstream_rdd_estimate_values = np.array(clickstream_rdd_estimate_value_list)
        np_clickstream_rdd_estimate_values_2 = np_clickstream_rdd_estimate_values**2
        np_clickstream_rdd_estimate_std_errors = np.array(clickstream_rdd_estimate_std_error_list)
        np_clickstream_rdd_estimate_variance = np_clickstream_rdd_estimate_std_errors**2

        # assign equal weights to each estimate
        clickstream_np_weights = np.array([1/len(clickstream_rdd_estimate_value_list)] * len(clickstream_rdd_estimate_value_list))

        # final_estimate_value = np.sum(np.array(rdd_estimate_value_list)*np_optimal_weights)
        final_clickstream_estimate_value = np.sum(np_clickstream_rdd_estimate_values * clickstream_np_weights) 
        final_clickstream_estimate_variance =  np.sum(np_clickstream_rdd_estimate_variance * clickstream_np_weights)\
        + np.sum(np_clickstream_rdd_estimate_values_2 * clickstream_np_weights)\
        - np.square(final_clickstream_estimate_value)
        final_clickstream_estimate_std_error = np.sqrt(final_clickstream_estimate_variance)

        final_clickstream_z_score = final_clickstream_estimate_value / final_clickstream_estimate_std_error
        final_clickstream_statistical_significance_p05 = np.abs(final_clickstream_z_score) >= 1.96
        final_clickstream_statistical_significance_p10 = np.abs(final_clickstream_z_score) >= 1.645
        print("(CLICKSTREAM) FINAL RDD ESTIMATE FOR: " + str(final_clickstream_estimate_value))
        print("(CLICKSTREAM) FINAL RDD ESTIMATE STD ERROR FOR " + str(final_clickstream_estimate_std_error))
        print("(CLICKSTREAM) FINAL Z SCORE FOR " + first_layer_index + " IS: " + str(final_clickstream_z_score))
        print("(CLICKSTREAM) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 5% FOR " + first_layer_index + " IS: " + str(final_clickstream_statistical_significance_p05))
        print("(CLICKSTREAM) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 10% FOR " + first_layer_index + " IS: " + str(final_clickstream_statistical_significance_p10))
        
    finalized_clickstream_rdd_estimates = {}
    finalized_clickstream_rdd_estimates["Estimate"] = final_clickstream_estimate_value
    finalized_clickstream_rdd_estimates["StdError"] = final_clickstream_estimate_std_error
    finalized_clickstream_rdd_estimates["ZScore"] = final_clickstream_z_score
    finalized_clickstream_rdd_estimates["Significance_5_perc"] = final_clickstream_statistical_significance_p05
    finalized_clickstream_rdd_estimates["Significance_10_perc"] = final_clickstream_statistical_significance_p10 
    
    
    print("--- TOTAL RUNTIME OF CLICKSTREAM RDD RESULT SUMMARY FOR" + first_layer_index + "IS " +  str(time.time() - start_time_00))
    
    return finalized_clickstream_rdd_estimates

In [19]:
# 14. Generate pd_valid_clickstream_rdd_estimates
def generate_pd_valid_clickstream_rdd_estimates(finalized_clickstream_rdd_estimates):
    
    k = 0

    for key, value in finalized_clickstream_rdd_estimates.items():

        key_list = key.split("_")
        Product = key_list[0]
        Year = key_list[1]
        Month = key_list[2]

        Estimate = value["Estimate"]
        StdError = value["StdError"]
        ZScore = value["ZScore"]
        Significance_5_perc = value["Significance_5_perc"]
        Significance_10_perc = value["Significance_10_perc"]

        if Significance_10_perc == True:
            if k == 0:
                valid_rdd_estimates_data = [[Product, Year, Month, Estimate, StdError]]
            else:
                valid_rdd_estimates_data = valid_rdd_estimates_data +  [[Product, Year, Month, Estimate, StdError]]

            k = k + 1

    pd_valid_rdd_estimates = pd.DataFrame(valid_rdd_estimates_data, columns = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH", "ESTIMATE", "STD_ERROR"])
    pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"] = pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"].astype(str)
    return pd_valid_rdd_estimates    

In [20]:
# 15. Generate pd_valid_clickstream_rdd_estimates
def generate_pd_valid_clickstream_rdd_estimates(finalized_clickstream_rdd_estimates):
    
    k = 0

    for key, value in finalized_clickstream_rdd_estimates.items():

        key_list = key.split("_")
        Product = key_list[0]
        Year = key_list[1]
        Month = key_list[2]

        Estimate = value["Estimate"]
        StdError = value["StdError"]
        ZScore = value["ZScore"]
        Significance_5_perc = value["Significance_5_perc"]
        Significance_10_perc = value["Significance_10_perc"]

        if Significance_10_perc == True:
            if k == 0:
                valid_rdd_estimates_data = [[Product, Year, Month, Estimate, StdError]]
            else:
                valid_rdd_estimates_data = valid_rdd_estimates_data +  [[Product, Year, Month, Estimate, StdError]]

            k = k + 1

    pd_valid_rdd_estimates = pd.DataFrame(valid_rdd_estimates_data, columns = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH", "ESTIMATE", "STD_ERROR"])
    pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"] = pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"].astype(str)
    return pd_valid_rdd_estimates

In [21]:
# 16. Generate time values & product x time values
def generate_clickstream_productxtime_sets(year_list, month_list, product_list, pd_sku_features):
    
    # 3.2.1. time values 
    time_values = [[m1, m2] for m1 in year_list for m2 in month_list]
    pd_time_values = pd.DataFrame(time_values, columns = ["YEAR", "MONTH"])
    pd_time_values["YEAR"] = pd_time_values["YEAR"].astype(str)
    pd_time_values["MONTH"] = pd_time_values["MONTH"].astype(str)

    # 3.2.2. product-time values 
    product_x_time_values = [[m1,m2,m3] for m1 in product_list for m2 in year_list for m3 in month_list]
    pd_product_x_time_values = pd.DataFrame(product_x_time_values, columns = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH"])
    pd_product_x_time_values["PRODUCT_PART_NUMBER"] = pd_product_x_time_values["PRODUCT_PART_NUMBER"].astype(str)
    pd_product_x_time_values["YEAR"] = pd_product_x_time_values["YEAR"].astype(str)
    pd_product_x_time_values["MONTH"] = pd_product_x_time_values["MONTH"].astype(str)
    
    # 3.2.3. Add features
    pd_product_x_time_with_sku_features = pd_product_x_time_values.merge(pd_sku_features, on = ["PRODUCT_PART_NUMBER"], how = "left")
    
    return pd_time_values, pd_product_x_time_with_sku_features 

In [22]:
# 17. Remediation: Generate final remediated estimates

def generate_clickstream_remediated_estimates(year_list, month_list, pd_time_values, pd_valid_rdd_estimates_with_sku_features, pd_product_x_time_with_sku_features):

    k=0
    for year in year_list:
        for month in month_list:

            # Get the slice that spans month 0 to the current month 
            pd_time_values_month0_to_current_month = pd_time_values.iloc[0:(k+1),:]
            pd_valid_rdd_estimates_with_sku_features_seed = pd_valid_rdd_estimates_with_sku_features.merge(pd_time_values_month0_to_current_month, on = ["YEAR", "MONTH"])

            # Get the slice that covers the current month (for constructing the complete results for the current month)
            pd_time_values_current_month = pd_time_values.iloc[k:(k+1),:]
            pd_valid_rdd_estimates_with_sku_features_current_month = pd_valid_rdd_estimates_with_sku_features.merge(pd_time_values_current_month, on = ["YEAR", "MONTH"])

            # Get the slice that covers the current month (for constructing the complete results for the current month – serving as left side)
            pd_product_x_time_with_sku_features_current_month =  pd_product_x_time_with_sku_features.merge(pd_time_values_current_month, on = ["YEAR", "MONTH"])

            # Summarize by SKU
            avg_estimate_by_SKU = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["PRODUCT_PART_NUMBER"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_SKU.columns = ["PRODUCT_PART_NUMBER", "SKU_AVG_ESTIMATE"]

            # Summarize by MC1-MC2-MC3 combination
            avg_estimate_by_MC1_MC2_MC3 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1","MC2","MC3"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1_MC2_MC3.columns = ["MC1","MC2","MC3", "MC1_MC2_MC3_AVG_ESTIMATE"]

            # Summarize by MC1-MC2 combination
            avg_estimate_by_MC1_MC2 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1", "MC2"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1_MC2.columns = ["MC1", "MC2", "MC1_MC2_AVG_ESTIMATE"]

            # Summarize by MC1
            avg_estimate_by_MC1 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1.columns = ["MC1", "MC1_AVG_ESTIMATE"]

            # Overall estimate
            avg_estimate_overall = np.mean(pd_valid_rdd_estimates_with_sku_features_seed["ESTIMATE"])

            # Now we merge the average results with current month results to complete remediation 
            matching_vars = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH", "MC1", "MC2", "MC3"]
            pd_product_x_time_with_sku_features_current_month_remediated = pd_product_x_time_with_sku_features_current_month.merge(pd_valid_rdd_estimates_with_sku_features_current_month, on = matching_vars, how = "left")\
            .merge(avg_estimate_by_SKU, on = ["PRODUCT_PART_NUMBER"], how = "left")\
            .merge(avg_estimate_by_MC1_MC2_MC3, on = ["MC1", "MC2", "MC3"], how = "left")\
            .merge(avg_estimate_by_MC1_MC2, on = ["MC1", "MC2"], how = "left")\
            .merge(avg_estimate_by_MC1, on = ["MC1"], how = "left")
            pd_product_x_time_with_sku_features_current_month_remediated["AVERAGE_ESTIMATE_OVERALL"] = avg_estimate_overall

            # Fill in missing values in the order of organic estimate, SKU average, MC1-MC2-MC3 average, MC1-MC2 average, M1 average, overall average 
            pd_product_x_time_with_sku_features_current_month_remediated["REMEDIATED_ESTIMATE"] = np.where(~pd_product_x_time_with_sku_features_current_month_remediated["ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["ESTIMATE"],
                                                                                                          np.where(~pd_product_x_time_with_sku_features_current_month_remediated["SKU_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["SKU_AVG_ESTIMATE"],
                                                                                                                  np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_MC3_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_MC3_AVG_ESTIMATE"],
                                                                                                                          np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_AVG_ESTIMATE"], 
                                                                                                                                  np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_AVG_ESTIMATE"], pd_product_x_time_with_sku_features_current_month_remediated["AVERAGE_ESTIMATE_OVERALL"])))))


            # Construct the complete estimates from month 0 to current month
            if k == 0:
                pd_remediated_estimates = pd_product_x_time_with_sku_features_current_month_remediated		
            else:
                pd_remediated_estimates = pd_remediated_estimates.append(pd_product_x_time_with_sku_features_current_month_remediated)


            k=k+1
            
    pd_remediated_estimates = pd_remediated_estimates.reset_index()             
    
    return pd_remediated_estimates

In [23]:
# 18. AS orders slice
def as_orders_slice_pandas(df_as_orders_data, sample_sku, sample_year, sample_month):
    
    df_as_orders_data_slice = df_as_orders_data.where((F.col("PRODUCT_PART_NUMBER")== str(sample_sku))\
                                                & (F.col("ORDER_PLACED_YEAR") == sample_year) \
                                                & (F.col("ORDER_PLACED_MONTH") == sample_month))
  
    pd_as_orders_data_slice = df_as_orders_data_slice.toPandas()
    
    return df_as_orders_data_slice, pd_as_orders_data_slice


In [24]:
# 19. AS orders count summaries

def as_orders_by_instock_status(pd_as_orders_data_slice):
    
    as_order_identifier = ["CUSTOMER_ID", 
                       "ORDER_ID",
                       "ORDER_LINE_ID",
                       "ORDER_PLACED_DTTM",
                        "PRODUCT_PART_NUMBER", 
                        "ORDER_ITEM_BACKORDER_FLAG", 
                        "ORDER_PLACED_YEAR", "ORDER_PLACED_MONTH"]

    is_as_orders_counts = len(pd_as_orders_data_slice[pd_as_orders_data_slice["ORDER_ITEM_BACKORDER_FLAG"] == False][as_order_identifier].drop_duplicates())
    oos_as_orders_counts = len(pd_as_orders_data_slice[pd_as_orders_data_slice["ORDER_ITEM_BACKORDER_FLAG"] == True][as_order_identifier].drop_duplicates())
    total_as_orders_counts = len(pd_as_orders_data_slice[as_order_identifier].drop_duplicates())    

    return is_as_orders_counts, oos_as_orders_counts, total_as_orders_counts


In [25]:
# 20. Find as orders cutoff dates

def find_as_orders_cutoff_dates(pd_as_orders_data_slice):

    # 1. Dummy variables that indicate IS and OOS
    
    pd_as_orders_data_slice["IS"] = np.where(pd_as_orders_data_slice["ORDER_ITEM_BACKORDER_FLAG"] == False, 1, 0)
    pd_as_orders_data_slice["OOS"] = np.where(pd_as_orders_data_slice["ORDER_ITEM_BACKORDER_FLAG"] == True, 1, 0)
    
    pd_as_orders_is_oos_counts_by_date = pd_as_orders_data_slice.groupby(["ORDER_PLACED_DATE"])[["IS", "OOS"]].sum().reset_index()
    pd_as_orders_is_oos_counts_by_date["IS_ADJ"] = np.where(pd_as_orders_is_oos_counts_by_date["IS"]/(pd_as_orders_is_oos_counts_by_date["IS"]+pd_as_orders_is_oos_counts_by_date["OOS"])<0.1, 0, pd_as_orders_is_oos_counts_by_date["IS"]) 
    pd_as_orders_is_oos_counts_by_date["OOS_ADJ"] = np.where(pd_as_orders_is_oos_counts_by_date["OOS"]/(pd_as_orders_is_oos_counts_by_date["IS"]+pd_as_orders_is_oos_counts_by_date["OOS"])<0.1, 0, pd_as_orders_is_oos_counts_by_date["OOS"]) 
    
    # 2. Generate status switching variable

    N = len(pd_as_orders_is_oos_counts_by_date)

    for row in range(N):

        if row == 0:

            temp = pd_as_orders_is_oos_counts_by_date.iloc[row:(row+1),].reset_index()
            is_counts = temp["IS_ADJ"][0]
            oos_counts = temp["OOS_ADJ"][0]
            
            # initialization for the first date in this monthly sample 
            if (is_counts == 0) & (oos_counts == 0):
                temp_status = "IS"
            elif (is_counts > 0) & (oos_counts == 0):
                temp_status = "IS"
            elif (is_counts == 0) & (oos_counts > 0):
                temp_status = "OOS"
            elif (is_counts > 0) & (oos_counts > 0):
                temp_status = "SWITCHING TO IS"

            temp["SWITCHING_STATUS"] = temp_status

            pd_as_orders_cutoff_point_detection = temp

        else:

            temp_t_minus_1 = pd_as_orders_cutoff_point_detection.iloc[(row-1):row,].reset_index()
            switching_status_t_minus_1 = temp_t_minus_1["SWITCHING_STATUS"][0]

            temp_t = pd_as_orders_is_oos_counts_by_date.iloc[row:(row+1),].reset_index()
            is_counts = temp_t["IS_ADJ"][0]
            oos_counts = temp_t["OOS_ADJ"][0]
            
            if (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "SWITCHING TO IS"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"
            elif (is_counts > 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"

            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "SWITCHING TO IS"    
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO IS"
            elif (is_counts > 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS"

            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "OOS"    
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "OOS"
            elif (is_counts == 0) & (oos_counts > 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "OOS"

            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "IS"):
                switching_status_t = "IS"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "OOS"):
                switching_status_t = "OOS"    
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO IS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO IS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"
            elif (is_counts == 0) & (oos_counts == 0) & (switching_status_t_minus_1 == "SWITCHING TO OOS (NON-CUTOFF)"):
                switching_status_t = "SWITCHING TO OOS (NON-CUTOFF)"

            else:
                switching_status_t = np.nan
                print("There is NULL value in the table.")

            temp_t["SWITCHING_STATUS"] = switching_status_t
            
            pd_as_orders_cutoff_point_detection = pd_as_orders_cutoff_point_detection.append(temp_t)

    pd_as_orders_cutoff_points = pd_as_orders_cutoff_point_detection[pd_as_orders_cutoff_point_detection["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"])][["ORDER_PLACED_DATE","SWITCHING_STATUS"]]    
    
    return pd_as_orders_cutoff_point_detection, pd_as_orders_cutoff_points



In [26]:
# 21. Scheduled AS orders RDD customers

def find_as_orders_rdd_relevant_customers(pd_as_orders_cutoff_points, pd_as_orders_cutoff_point_detection, pd_as_orders_data_slice, sufficiency_ratio, initial_window_length, first_layer_index):
     
    # print("==========CONSTRUCTING AS_ORDERS RDD CUSTOMER SAMPLE FOR " + first_layer_index + " ==========")

    # as_orders_rdd_customers_temp = pd.DataFrame(columns = ["CUSTOMER_ID"])

    as_orders_rdd_customers_temp = []
    
    # 1. RDD sample generation
    for k in range(len(pd_as_orders_cutoff_points)):

        cutoff_point = pd_as_orders_cutoff_points.iloc[k,]["ORDER_PLACED_DATE"]

        cutoff_point_name = pd_as_orders_cutoff_points.iloc[k,]["SWITCHING_STATUS"]

        window_length = initial_window_length

        # Check T days prior to the cutoff date and T days after the cutoff date
        while window_length > 0:

            # Window start date
            window_left_end = max(cutoff_point - DT.timedelta(days=window_length), min(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"]))

            # Window end date
            window_right_end = min(cutoff_point + DT.timedelta(days=(window_length+1)), max(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"])+ DT.timedelta(days=1))

            pd_as_orders_cutoff_point_detection_rdd_sample_temp = pd_as_orders_cutoff_point_detection[(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"] >= window_left_end) & (pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"] < window_right_end) ]

            # [1] We want to make sure this sample only has the cutoff point that is under inspection
            if np.sum(pd_as_orders_cutoff_point_detection_rdd_sample_temp["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"]))>1:
                window_length = window_length - 1
                if window_length == 0:
                    # print("THIS IS THE SHORTEST AS ORDERS RDD SAMPLE OF " + str(cutoff_point) + " YET IT IS STILL NOT CLEAN")
                    temp = 1
                continue
            else:
                # print("FOUND A CLEAN AS ORDERS RDD SAMPLE FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("CHECKING DATA SUFFICIENCY")
                temp = 1

            # [1] AS ORDERS RDD horizon
            as_orders_rdd_sample_temp = pd_as_orders_data_slice[ (pd_as_orders_data_slice["ORDER_PLACED_DATE"]>=window_left_end) & (pd_as_orders_data_slice["ORDER_PLACED_DATE"]< window_right_end)]

            # [2] Remove duplicate clickstream data points   
            as_orders_identifier_variables = ["CUSTOMER_ID", 
                           "ORDER_ID",
                           "ORDER_LINE_ID",
                           "ORDER_PLACED_DTTM",
                            "PRODUCT_PART_NUMBER", 
                            "ORDER_ITEM_BACKORDER_FLAG", 
                            "ORDER_STATUS",
                            "ORDER_PLACED_DATE", "ORDER_PLACED_YEAR", "ORDER_PLACED_MONTH"]
            as_orders_rdd_sample_cleaned = as_orders_rdd_sample_temp[as_orders_identifier_variables].drop_duplicates()

            # [3] Ensure at least 50% of days on both sides of the cutoff point need to have data 
            # Summarize daily data points
            as_orders_counts_distribution_across_days = as_orders_rdd_sample_cleaned.groupby(["ORDER_PLACED_DATE"])["CUSTOMER_ID"].count().reset_index()

            if (as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] < cutoff_point]["ORDER_PLACED_DATE"].nunique() >= window_length*sufficiency_ratio) & \
                (as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] > cutoff_point]["ORDER_PLACED_DATE"].nunique() >= window_length*sufficiency_ratio):
                # print("WE FOUND SUFFICIENT DATA FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("LEFT SIDE HAS " + str(as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] < cutoff_point]["ORDER_PLACED_DATE"].nunique()) + " DAYS OF DATA")
                # print("RIGHT SIDE HAS " + str(as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] > cutoff_point]["ORDER_PLACED_DATE"].nunique()) + " DAYS OF DATA")
                # print("THIS RDD SAMPLE IS COLLECTED FOR REGRESSION AND ANALYSES")
                temp = 1
                break

            else:
                window_length = window_length - 1
                if window_length == 0:   
                    # print("THIS IS THE SHORTEST AS ORDERS RDD SAMPLE OF " + str(cutoff_point) + " YET IT DOES NOT HAVE SUFFICIENT DATA")
                    temp = 1
                continue

        if window_length == 0:
            # print("UNFORTUNATELY, WE CANNOT FIND AN IDEAL AS ORDERS RDD SAMPLE FOR CUTOFF POINT: " + str(cutoff_point))
            temp = 1
            continue

        else:
            # print("WE HAVE FOUND THE RDD SAMPLE FOR REGRESSION FOR CUTOFF POINT: " + str(cutoff_point))
            # print("THE LEFT END DATE IS: " + str(window_left_end))
            # print("THE RIGHT END DATE IS: " + str(window_right_end))
            temp = 1

            # [1] Collect the customers

            as_orders_rdd_customers_temp0 = as_orders_rdd_sample_cleaned[["CUSTOMER_ID"]].drop_duplicates()["CUSTOMER_ID"].to_list()

            second_layer_index = str(cutoff_point) + "_" + str(cutoff_point_name) + "_" + str(window_length)

            # print("(AS_ORDERS) THE NUMBER OF CUSTOMERS IN " + first_layer_index + " SAMPLE " + second_layer_index + " IS " + str(len(as_orders_rdd_customers_temp0)))

            # as_orders_rdd_customers_temp = as_orders_rdd_customers_temp.append(as_orders_rdd_customers_temp0).drop_duplicates().reset_index()[["CUSTOMER_ID"]]
    
            as_orders_rdd_customers_temp = as_orders_rdd_customers_temp + as_orders_rdd_customers_temp0
            as_orders_rdd_customers_temp = [*set(as_orders_rdd_customers_temp)]
        
    return as_orders_rdd_customers_temp

In [27]:
# 22. AS Orders RDD sample construction
def construct_as_orders_rdd_samples(pd_as_orders_cutoff_points, pd_as_orders_cutoff_point_detection, pd_as_orders_data_slice, sufficiency_ratio, initial_window_length, first_layer_index, df_cp):
     
    print("==========CONSTRUCTING AS ORDERS RDD SAMPLE FOR " + first_layer_index + " ==========")
    start_time_00 = time.time()
    
    as_orders_rdd_sample_collection_temp = {}
    
    # 1. RDD sample generation
    for k in range(len(pd_as_orders_cutoff_points)):

        cutoff_point = pd_as_orders_cutoff_points.iloc[k,]["ORDER_PLACED_DATE"]

        cutoff_point_name = pd_as_orders_cutoff_points.iloc[k,]["SWITCHING_STATUS"]

        window_length = initial_window_length
	
        # Check T days prior to the cutoff date and T days after the cutoff date
        while window_length > 0:

            # Window start date
            window_left_end = max(cutoff_point - DT.timedelta(days=window_length), min(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"]))

            # Window end date
            window_right_end = min(cutoff_point + DT.timedelta(days=(window_length+1)), max(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"])+ DT.timedelta(days=1))

            pd_as_orders_cutoff_point_detection_rdd_sample_temp = pd_as_orders_cutoff_point_detection[(pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"] >= window_left_end) & (pd_as_orders_cutoff_point_detection["ORDER_PLACED_DATE"] < window_right_end) ]

            # [1] We want to make sure this sample only has the cutoff point that is under inspection
            if np.sum(pd_as_orders_cutoff_point_detection_rdd_sample_temp["SWITCHING_STATUS"].isin(["SWITCHING TO IS", "SWITCHING TO OOS"]))>1:
                window_length = window_length - 1
                if window_length == 0:
                    # print("THIS IS THE SHORTEST AS ORDERS RDD SAMPLE OF " + str(cutoff_point) + " YET IT IS STILL NOT CLEAN")
                    temp = 1
                continue
            else:
                # print("FOUND A CLEAN AS ORDERS RDD SAMPLE FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("CHECKING DATA SUFFICIENCY")
                temp = 1
                
            # [1] AS ORDERS RDD horizon
            as_orders_rdd_sample_temp = pd_as_orders_data_slice[ (pd_as_orders_data_slice["ORDER_PLACED_DATE"]>=window_left_end) & (pd_as_orders_data_slice["ORDER_PLACED_DATE"]< window_right_end)]

            # [2] Remove duplicate clickstream data points   
            as_orders_identifier_variables = ["CUSTOMER_ID", 
                           "ORDER_ID",
                           "ORDER_LINE_ID",
                           "ORDER_PLACED_DTTM",
                            "PRODUCT_PART_NUMBER", 
                            "ORDER_ITEM_BACKORDER_FLAG", 
                            "ORDER_STATUS",
                            "ORDER_PLACED_DATE", "ORDER_PLACED_YEAR", "ORDER_PLACED_MONTH"]
            as_orders_rdd_sample_cleaned = as_orders_rdd_sample_temp[as_orders_identifier_variables].drop_duplicates()

            # [3] Ensure at least 50% of days on both sides of the cutoff point need to have data 
            # Summarize daily data points
            as_orders_counts_distribution_across_days = as_orders_rdd_sample_cleaned.groupby(["ORDER_PLACED_DATE"])["CUSTOMER_ID"].count().reset_index()

            if (as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] < cutoff_point]["ORDER_PLACED_DATE"].nunique() >= window_length*sufficiency_ratio) & \
                (as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] > cutoff_point]["ORDER_PLACED_DATE"].nunique() >= window_length*sufficiency_ratio):
                # print("WE FOUND SUFFICIENT DATA FOR " + str(cutoff_point) + " WITH WINDOW LENGTH " + str(window_length))
                # print("LEFT SIDE HAS " + str(as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] < cutoff_point]["ORDER_PLACED_DATE"].nunique()) + " DAYS OF DATA")
                # print("RIGHT SIDE HAS " + str(as_orders_counts_distribution_across_days[as_orders_counts_distribution_across_days["ORDER_PLACED_DATE"] > cutoff_point]["ORDER_PLACED_DATE"].nunique()) + " DAYS OF DATA")
                # print("THIS RDD SAMPLE IS COLLECTED FOR REGRESSION AND ANALYSES")
                temp = 1
                break

            else:
                window_length = window_length - 1
                if window_length == 0:   
                    # print("THIS IS THE SHORTEST AS ORDERS RDD SAMPLE OF " + str(cutoff_point) + " YET IT DOES NOT HAVE SUFFICIENT DATA")
                    temp = 1
                continue

        if window_length == 0:
            # print("UNFORTUNATELY, WE CANNOT FIND AN IDEAL AS ORDERS RDD SAMPLE FOR CUTOFF POINT: " + str(cutoff_point))
            temp = 1
            continue

        else:
            # print("WE HAVE FOUND THE RDD SAMPLE FOR REGRESSION FOR CUTOFF POINT: " + str(cutoff_point))
            # print("THE LEFT END DATE IS: " + str(window_left_end))
            # print("THE RIGHT END DATE IS: " + str(window_right_end))
            temp = 1
            
            # 2. Feature Engineering

            # [1] The precise timestamp of the cutoff point
            
            # We make the following update: if the status switches from OOS to IS then we use the first IS timestamp within the cutoff date as the cutoff timestamp. Similar for IS -> OOS.  
            
            as_orders_rdd_sample_on_cutoff_date = as_orders_rdd_sample_cleaned[as_orders_rdd_sample_cleaned["ORDER_PLACED_DATE"] == cutoff_point]
           
            as_orders_rdd_sample_on_cutoff_date["STATUS_RANK"] = as_orders_rdd_sample_on_cutoff_date.groupby("ORDER_ITEM_BACKORDER_FLAG")["ORDER_PLACED_DTTM"].rank(method = "dense", ascending = True).astype(int)

            if cutoff_point_name == "SWITCHING TO IS":

                cutoff_point_timestamp = as_orders_rdd_sample_on_cutoff_date[as_orders_rdd_sample_on_cutoff_date["STATUS_RANK"]==1][as_orders_rdd_sample_on_cutoff_date["ORDER_ITEM_BACKORDER_FLAG"] == False].reset_index().iloc[0]["ORDER_PLACED_DTTM"]

            else:

                cutoff_point_timestamp = as_orders_rdd_sample_on_cutoff_date[as_orders_rdd_sample_on_cutoff_date["STATUS_RANK"]==1][as_orders_rdd_sample_on_cutoff_date["ORDER_ITEM_BACKORDER_FLAG"] == True].reset_index().iloc[0]["ORDER_PLACED_DTTM"]

            # [2] Treatment variable    
            
            as_orders_rdd_sample_cleaned["TREATMENT_VARIABLE"] = np.where(as_orders_rdd_sample_cleaned["ORDER_ITEM_BACKORDER_FLAG"] == False, 1, 0)

            # [3] Generate the hour difference between the timestamp and the cutoff point
            
            # We make the following update: We are going to drop data points that are "not consistent with RDD design"
           
            as_orders_rdd_sample_cleaned["RUNNING_VARIABLE"] = (as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM"] - cutoff_point_timestamp).dt.total_seconds()/3600

            if cutoff_point_name == "SWITCHING TO IS":

                as_orders_rdd_sample_cleaned["DROP_INDICATOR"] = np.where( ((as_orders_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 1) & (as_orders_rdd_sample_cleaned["RUNNING_VARIABLE"] < 0)) \
                                                                              | ((as_orders_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 0) & (as_orders_rdd_sample_cleaned["RUNNING_VARIABLE"] >= 0)), 1, 0)
          
                # print("BEFORE DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(as_orders_rdd_sample_cleaned)))
                
                as_orders_rdd_sample_cleaned = as_orders_rdd_sample_cleaned[as_orders_rdd_sample_cleaned["DROP_INDICATOR"]==0]
                
                # print("AFTER DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(as_orders_rdd_sample_cleaned)))
                
            else:
                
                as_orders_rdd_sample_cleaned["DROP_INDICATOR"] = np.where( ((as_orders_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 1) & (as_orders_rdd_sample_cleaned["RUNNING_VARIABLE"] >= 0)) \
                                                                              | ((as_orders_rdd_sample_cleaned["TREATMENT_VARIABLE"] == 0) & (as_orders_rdd_sample_cleaned["RUNNING_VARIABLE"] < 0)), 1, 0)
          
                # print("BEFORE DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(as_orders_rdd_sample_cleaned)))
                
                as_orders_rdd_sample_cleaned = as_orders_rdd_sample_cleaned[as_orders_rdd_sample_cleaned["DROP_INDICATOR"]==0]
                
                # print("AFTER DROPPING THE INCONSISTENT DATA POINTS, # OF DATA POINTS IS: " + str(len(as_orders_rdd_sample_cleaned)))
            
            # [4] Outcome Variable Construction

            customer_id_list_in_as_orders_rdd_sample = as_orders_rdd_sample_cleaned[["CUSTOMER_ID"]].drop_duplicates()["CUSTOMER_ID"].to_list()

            # Add filtering variables
            as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM_UB"] = as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM"] + pd.Timedelta(hours=24)
            as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM_END"] = as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM"] + pd.Timedelta(hours=24*365)

            # Max timestamp possible for CP calculation
            max_timestamp = np.max(as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM_END"])
            # Min timestamp possible for CP calculation
            min_timestamp = np.min(as_orders_rdd_sample_cleaned["ORDER_PLACED_DTTM"])

            # Focus on customers in clickstream data & Focus on time period that is relevant
            df_cp_as_orders = df_cp.filter( (F.col("CUSTOMER_ID").isin(customer_id_list_in_as_orders_rdd_sample)) & \
                                            (F.col("ORDER_PLACED_DTTM") >= min_timestamp) & \
                                            (F.col("ORDER_PLACED_DTTM") <= max_timestamp) )
            # df_cp_as_orders = df_cp_as_orders.na.fill(value=0,subset=["IB_COST", "OB_COST", "COGS", "AVERAGE_PRICE"])
            # df_cp_as_orders = df_cp_as_orders.withColumn("NET_MARGIN", (F.col("AVERAGE_PRICE")-F.col("IB_COST")-F.col("OB_COST")-F.col("COGS"))*F.col("ORDER_LINE_QUANTITY") )

            df_cp_as_orders = df_cp_as_orders.withColumn("NET_MARGIN_V2",  F.col("AVERAGE_PRICE")-F.col("IB_COST")-F.col("OB_COST")-F.col("COGS"))

            # Transform the small cp data into pandas dataframe
            print("==========Converting CP data into Pandas data frame==========")
            
            start_time = time.time()
            pd_cp_as_orders = df_cp_as_orders.toPandas()
            pd_cp_as_orders["ORDER_LINE_QUANTITY"] = pd_cp_as_orders["ORDER_LINE_QUANTITY"].astype(float)
            # CP data construction: keep the relevant columns
            cp_relevant_columns = ["CUSTOMER_ID", "PRODUCT_PART_NUMBER_IN_CP", "ORDER_PLACED_DTTM_CP", "ORDER_LINE_QUANTITY", "IS_AUTO_SHIP", "NET_MARGIN", "NET_MARGIN_V2"]
            pd_cp_as_orders_adj = pd_cp_as_orders[cp_relevant_columns]
            
            print("--- %s seconds ---" % (time.time() - start_time))

            # Keep the relevant columns of scheduled as orders data
            as_orders_relevant_columns = as_orders_identifier_variables + ["RUNNING_VARIABLE", "TREATMENT_VARIABLE","ORDER_PLACED_DTTM_UB", "ORDER_PLACED_DTTM_END"]
            as_orders_rdd_sample_cleaned_adj = as_orders_rdd_sample_cleaned[as_orders_relevant_columns]

            start_time_0 = time.time()

            # alternative way
            for row in range(len(as_orders_rdd_sample_cleaned_adj)):

                # print("==========NOW GENERATING CP FOR DATA POINT: " + str(row) + " ==========")

                # start_time = time.time()

                # Loop through each as order data point

                as_order_dp = as_orders_rdd_sample_cleaned_adj.iloc[row:(row+1),:].reset_index()

                # info needed to be added to CP data
                customer_id = as_order_dp["CUSTOMER_ID"][0]
                order_id = as_order_dp["ORDER_ID"][0]
                order_line_id = as_order_dp["ORDER_LINE_ID"][0]
                product_part_number = as_order_dp["PRODUCT_PART_NUMBER"][0]
                timestamp_lb = as_order_dp["ORDER_PLACED_DTTM"][0]
                timestamp_ub = as_order_dp["ORDER_PLACED_DTTM_UB"][0]
                timestamp_end = as_order_dp["ORDER_PLACED_DTTM_END"][0]
                order_date = as_order_dp["ORDER_PLACED_DATE"][0]
                order_year = as_order_dp["ORDER_PLACED_YEAR"][0]
                order_month = as_order_dp["ORDER_PLACED_MONTH"][0]
                instock_status = as_order_dp["ORDER_ITEM_BACKORDER_FLAG"][0]
                order_status = as_order_dp["ORDER_STATUS"][0]
                running_variable = as_order_dp["RUNNING_VARIABLE"][0]
                treatment_variable = as_order_dp["TREATMENT_VARIABLE"][0]

                # get CP data that corresponds to the customer id of the clickstream dp
                pd_cp_as_orders_dp = pd_cp_as_orders_adj[pd_cp_as_orders_adj["CUSTOMER_ID"] == customer_id]

                # add the info from the as order dp into CP data
                pd_cp_as_orders_dp["PRODUCT_PART_NUMBER"] = product_part_number
                pd_cp_as_orders_dp["ORDER_ID"] = order_id
                pd_cp_as_orders_dp["ORDER_LINE_ID"] = order_line_id
                pd_cp_as_orders_dp["ORDER_PLACED_DTTM"] = timestamp_lb
                pd_cp_as_orders_dp["ORDER_PLACED_DTTM_UB"] = timestamp_ub
                pd_cp_as_orders_dp["ORDER_PLACED_DTTM_END"] = timestamp_end
                pd_cp_as_orders_dp["ORDER_PLACED_DATE"] = order_date
                pd_cp_as_orders_dp["ORDER_PLACED_YEAR"] = order_year
                pd_cp_as_orders_dp["ORDER_PLACED_MONTH"] = order_month
                pd_cp_as_orders_dp["ORDER_ITEM_BACKORDER_FLAG"] = instock_status
                pd_cp_as_orders_dp["ORDER_STATUS"] = order_status
                pd_cp_as_orders_dp["RUNNING_VARIABLE"] = running_variable
                pd_cp_as_orders_dp["TREATMENT_VARIABLE"] = treatment_variable

                # if len(pd_cp_as_orders_dp) == 0:
                #    print("THIS DATA POINT HAS NO CP RECORD")

                # else:
                #    print("THIS DATA POINT HAS CP RECORD: " + str(len(pd_cp_as_orders_dp)))

                pd_cp_as_orders_dp["CP_INCLUDED"] = np.where(

                    ((pd_cp_as_orders_dp['PRODUCT_PART_NUMBER_IN_CP'] != pd_cp_as_orders_dp['PRODUCT_PART_NUMBER']) & (pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_as_orders_dp['ORDER_PLACED_DTTM']) & (pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_as_orders_dp['ORDER_PLACED_DTTM_UB']))\
                    | \

                    ((pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_as_orders_dp['ORDER_PLACED_DTTM_UB']) & (pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_as_orders_dp['ORDER_PLACED_DTTM_END']))

                    , 1, 0)                
                
                pd_cp_as_orders_dp["CP_INCLUDED_V2"] = np.where(

                (pd_cp_as_orders_dp['PRODUCT_PART_NUMBER_IN_CP'] != pd_cp_as_orders_dp['PRODUCT_PART_NUMBER']) \
                 & ((pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] >= pd_cp_as_orders_dp['ORDER_PLACED_DTTM']) \
                    & (pd_cp_as_orders_dp['ORDER_PLACED_DTTM_CP'] < pd_cp_as_orders_dp['ORDER_PLACED_DTTM_END']))

                , 1, 0)

                pd_cp_as_orders_dp["CP"] = pd_cp_as_orders_dp["CP_INCLUDED"] * pd_cp_as_orders_dp["NET_MARGIN"] * pd_cp_as_orders_dp["ORDER_LINE_QUANTITY"]
                pd_cp_as_orders_dp["CP_V2"] = pd_cp_as_orders_dp["CP_INCLUDED"] * pd_cp_as_orders_dp["NET_MARGIN_V2"] * pd_cp_as_orders_dp["ORDER_LINE_QUANTITY"]
                pd_cp_as_orders_dp["CP_V3"] = pd_cp_as_orders_dp["CP_INCLUDED_V2"] * pd_cp_as_orders_dp["NET_MARGIN"] * pd_cp_as_orders_dp["ORDER_LINE_QUANTITY"]
                pd_cp_as_orders_dp["CP_V4"] = pd_cp_as_orders_dp["CP_INCLUDED_V2"] * pd_cp_as_orders_dp["NET_MARGIN_V2"] * pd_cp_as_orders_dp["ORDER_LINE_QUANTITY"]
                
                groupVars = as_orders_relevant_columns

                pd_agg_cp_as_orders_dp = pd_cp_as_orders_dp.groupby(groupVars)[["CP", "CP_V2", "CP_V3", "CP_V4"]].sum().reset_index()

                if row == 0:

                    pd_as_orders_cp_outcome =  pd_agg_cp_as_orders_dp

                else:

                    pd_as_orders_cp_outcome = pd_as_orders_cp_outcome.append(pd_agg_cp_as_orders_dp)

                # print("==========GENERATION IS COMPLETED FOR DATA POINT: " + str(row) + " ==========")
                # print("--- %s seconds ---" % (time.time() - start_time))

            # print("--- TOTAL --- %s seconds ---" % (time.time() - start_time_0))

            as_orders_rdd_sample_cleaned_adj["CUSTOMER_ID"] = as_orders_rdd_sample_cleaned_adj["CUSTOMER_ID"].astype(str)
            pd_as_orders_cp_outcome["CUSTOMER_ID"] = pd_as_orders_cp_outcome["CUSTOMER_ID"].astype(str)
            rdd_sample = pd.merge(as_orders_rdd_sample_cleaned_adj, pd_as_orders_cp_outcome, on = groupVars, how = "left")
    
            second_layer_index = str(cutoff_point) + "_" + str(cutoff_point_name) + "_" + str(window_length)
            print("CP NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP"].isna())/len(rdd_sample)))
            print("CP_V2 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V2"].isna())/len(rdd_sample)))
            print("CP_V3 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V3"].isna())/len(rdd_sample)))
            print("CP_V4 NULL VALUE COUNTS FOR " + first_layer_index + "_" + second_layer_index + " IS " + str(sum(rdd_sample["CP_V4"].isna())/len(rdd_sample)))

            rdd_sample = rdd_sample.fillna({"CP":0,
                                           "CP_V2":0,
                                            "CP_V3":0,
                                            "CP_V4":0})

            as_orders_rdd_sample_collection_temp[second_layer_index] = rdd_sample
        
        
    print("--- TOTAL RUNTIME FOR RDD CONSTRUCTION OF" + first_layer_index + "IS " +  str(time.time() - start_time_00))
        
    return as_orders_rdd_sample_collection_temp





In [28]:
# 23. AS_ORDERS RDD
def as_orders_rdd_modeling(as_orders_rdd_sample_collection, first_layer_index, cp_variable, K, regression_type, h, a1, a2, bootstrapping_method, boostramp_sample_size):

    as_orders_rdd_estimate_collection = {}
    
    print("========== RUNNING AS_ORDERS RDD FOR " + first_layer_index + " ==========")
    
    start_time_00 = time.time()
    
    if regression_type == "SEPARATE_UNWEIGHTED":

        for sample_index, pd_rdd_sample in as_orders_rdd_sample_collection.items():
            print("========== NOW RUNNING SEPARATE_UNWEIGHTED REGRESSION FOR SAMPLE: " + str(sample_index) + "==========")

            print("DATA POINTS: " + str(len(pd_rdd_sample)))

            print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
            pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
            pd_rdd_sample

            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
            pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
            pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":
                
                plt.figure(0)

                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()
                
                
                plt.figure(1)
            
                # another version of the graph
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show()

            else:

                
                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()        
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show() 

            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL
            K = 1

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]


                results = sm.OLS(Y, X).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(AS_ORDERS) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:

                estimate_list = []
                for s in range(boostramp_sample_size):
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)
                    
                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.OLS(Y, X).fit()
                    
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                    
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(AS_ORDERS) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


            as_orders_rdd_estimate_collection[sample_index] = {}
            as_orders_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
            as_orders_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err


    elif regression_type == "SEPARATE_WEIGHTED":

        for sample_index, pd_rdd_sample in as_orders_rdd_sample_collection.items():
            print("========== NOW RUNNING SEPARATE_WEIGHTED REGRESSION FOR SAMPLE: " + str(sample_index) + "==========")

            print("DATA POINTS: " + str(len(pd_rdd_sample)))

            print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
            pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
            pd_rdd_sample

            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
            pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
            pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
            pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
            pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":

                
                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show()

            else:

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()      
                
                # another version of the graph
                
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")
                
                plt.show() 

            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]


            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(AS_ORDERS) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:
                
                estimate_list = []
                
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(AS_ORDERS) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


            as_orders_rdd_estimate_collection[sample_index] = {}
            as_orders_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
            as_orders_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err
		
    elif regression_type == "JOINT_WEIGHTED":

        index = 1
        
        for sample_index_temp, pd_rdd_sample_temp in as_orders_rdd_sample_collection.items():
            
            print("========== NOW RUNNING JOINT_WEIGHTED REGRESSION ==========")
            print("FOR SAMPLE " + sample_index_temp + ", DATA POINTS: " + str(len(pd_rdd_sample_temp)))

            pd_rdd_sample_temp["SAMPLE_INDEX"] = index

            if index == 1:

                pd_rdd_sample = pd_rdd_sample_temp
                sample_index = sample_index_temp

            else:

                pd_rdd_sample = pd_rdd_sample.append(pd_rdd_sample_temp)
                sample_index = sample_index_temp

            index = index + 1
            
            print("JOINT SAMPLE HAS DATA POINTS: " + str(len(pd_rdd_sample)))


        print("========== STEP 1: FURTHER FEATURE ENGINEERING: REMOVE NULL DATA POINTS ==========")  
        pd_rdd_sample = pd_rdd_sample[~pd_rdd_sample[cp_variable].isna()]
        pd_rdd_sample

        print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

        print("========== STEP 2: FURTHER FEATURE ENGINEERING: ADD NON-LINEARITY==========")
        pd_rdd_sample["RUNNING_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE"] 
        pd_rdd_sample["RUNNING_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE"] * pd_rdd_sample["RUNNING_VARIABLE"]
        pd_rdd_sample["RUNNING_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["RUNNING_VARIABLE"]
        pd_rdd_sample["RUNNING_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["RUNNING_VARIABLE"]

        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_1"] = pd_rdd_sample["RUNNING_VARIABLE_1"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_2"] = pd_rdd_sample["RUNNING_VARIABLE_2"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_3"] = pd_rdd_sample["RUNNING_VARIABLE_3"] * pd_rdd_sample["TREATMENT_VARIABLE"]
        pd_rdd_sample["RUNNING_x_TREATMENT_VARIABLE_4"] = pd_rdd_sample["RUNNING_VARIABLE_4"] * pd_rdd_sample["TREATMENT_VARIABLE"]



        if np.max(pd_rdd_sample["SAMPLE_INDEX"]) == 1:
            
            print("NO NEED TO JOIN AND PROCEED AS SEPARATE REGRESSION")

            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========") 
            
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: VISUALIZATION==========")
            if sample_index.split("_")[1] == "SWITCHING TO IS":

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()

                
                # another version of the graph
                plt.figure(1)
                
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')
                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")   
                
                plt.show()
                
            else:

                plt.figure(0)
                
                x1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)]["RUNNING_VARIABLE_1"]
                y1 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]<=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==1)][cp_variable]
                ax = sns.regplot(x1, y1,

                           scatter_kws = {"alpha": 0.33},

                           )

                x2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)]["RUNNING_VARIABLE_1"]
                y2 = pd_rdd_sample[(pd_rdd_sample["RUNNING_VARIABLE_1"]>=0) & (pd_rdd_sample["TREATMENT_VARIABLE"]==0)][cp_variable]
                ax = sns.regplot(x2, y2,

                            scatter_kws = {"alpha": 0.33},


                           )

                plt.axvline(0, min(pd_rdd_sample[cp_variable]), max(pd_rdd_sample[cp_variable]), c = "red",  linestyle="dashed")

                ax.set(xlabel = "X: Running Variable (Hour Relative to Cutoff Point)", ylabel = "Y: Contribution Profit ($)")

                ax.set(title = "RDD Visualization For " + first_layer_index + "_" + sample_index)

                plt.show()        
                
                # another version of the graph
                
                plt.figure(1)
                edges = np.linspace(-h,h,2*h+1)
                pd_rdd_sample['Bins'] = pd.cut(pd_rdd_sample['RUNNING_VARIABLE_1'], bins = edges)

                # Mean within bins
                binned = pd_rdd_sample.groupby(['Bins']).agg('mean')

                # And plot
                sns.lineplot(x = binned['RUNNING_VARIABLE_1'],
                          y = binned[cp_variable], marker = 'o')

                # Add vertical line at cutoff
                plt.axvline(0, min(binned[cp_variable]), max(binned[cp_variable]), c = "red",  linestyle="dashed")

                plt.show()
                
            print("========== STEP 6: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(AS_ORDERS) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:

                estimate_list = []
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(AS_ORDERS) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))


        else:
            
            index_var_name_list = []
            
            for m in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1):
                
                index_var_name = "SAMPLE_INDEX" + "_" + str(m)
                pd_rdd_sample[index_var_name] = np.where(pd_rdd_sample["SAMPLE_INDEX"] == m, 1, 0)
                index_var_name_list = index_var_name_list + [index_var_name]

            vars_for_interaction_with_sample_index = ["RUNNING_VARIABLE_1", "RUNNING_VARIABLE_2", "RUNNING_VARIABLE_3", "RUNNING_VARIABLE_4",
                                                      "RUNNING_x_TREATMENT_VARIABLE_1", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_2", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_3", 
                                                      "RUNNING_x_TREATMENT_VARIABLE_4"]
            
            interaction_var_name_list = []
            
            for m1 in index_var_name_list:
                for m2 in vars_for_interaction_with_sample_index:
                    
                    interaction_var_name = m2 + "_" + m1
                    pd_rdd_sample[interaction_var_name] = pd_rdd_sample[m1] * pd_rdd_sample[m2]
                    interaction_var_name_list = interaction_var_name_list + [interaction_var_name]


            print("========== STEP 3: FURTHER FEATURE ENGINEERING: WINDOW LENGTH ADJUSTMENT==========")   
            rdd_window_lb = min(pd_rdd_sample["RUNNING_VARIABLE"])
            rdd_window_ub = max(pd_rdd_sample["RUNNING_VARIABLE"])

            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample["RUNNING_VARIABLE"] >= rdd_window_lb) & (pd_rdd_sample["RUNNING_VARIABLE"] <= rdd_window_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 4: FILTERING OUT EXTREME CP VALUES==========")   
            lb_perc = .025
            ub_perc = .975
            perc_lb = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[lb_perc]
            perc_ub = pd_rdd_sample[cp_variable].quantile([lb_perc, ub_perc])[ub_perc]
            pd_rdd_sample = pd_rdd_sample[ (pd_rdd_sample[cp_variable] >= perc_lb) & (pd_rdd_sample[cp_variable] <= perc_ub) ]
            print("RESULTING DATA POINTS: " +str(len(pd_rdd_sample)))

            print("========== STEP 5: REGRESSION==========")

            # ORDER OF POLYNOMIAL

            var_list = ["TREATMENT_VARIABLE"] + ["RUNNING_VARIABLE_" + str(n) for n in np.arange(1,K+1)] \
            + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n) for n in np.arange(1,K+1)] \
            + ["RUNNING_VARIABLE_" + str(n1)  + "_" + "SAMPLE_INDEX_" + str(n2) for n1 in np.arange(1,K+1) for n2 in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1)] \
            + ["RUNNING_x_TREATMENT_VARIABLE_" + str(n1)  + "_" + "SAMPLE_INDEX_" + str(n2) for n1 in np.arange(1,K+1) for n2 in np.arange(2, np.max(pd_rdd_sample["SAMPLE_INDEX"])+1)]

            if bootstrapping_method == False:

                X = pd_rdd_sample[var_list]
                X = sm.add_constant(X)

                Y = pd_rdd_sample[[cp_variable]]

                results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()

                print(results.summary())

                rdd_estimate_value = results.params["TREATMENT_VARIABLE"]
                rdd_estimate_std_err = results.bse["TREATMENT_VARIABLE"]

                print("(AS_ORDERS) THE ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))

            else:
                
                estimate_list = []
                
                for s in range(boostramp_sample_size):
                    
                    pd_rdd_sample_resampled = pd_rdd_sample.sample(n = len(pd_rdd_sample), replace = True)
                    
                    X = pd_rdd_sample_resampled[var_list]
                    X = sm.add_constant(X)

                    Y = pd_rdd_sample_resampled[[cp_variable]]
                    
                    results = sm.WLS(Y,X, weights= kernel(X["RUNNING_VARIABLE_1"], h, a1, a2)).fit()
                    estimate_list = estimate_list + [results.params["TREATMENT_VARIABLE"]]
                    
                plt.hist(np.array(estimate_list))
                plt.show()
                
                rdd_estimate_value = np.mean(np.array(estimate_list))
                rdd_estimate_std_err = np.std(np.array(estimate_list))
                
                print("(AS_ORDERS) THE BOOSTRAPPED ESTIMATED TREATMENT EFFECT FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_value))
                print("(AS_ORDERS) THE BOOSTRAPPED STANDARD ERROR OF THE ESTIMATE FOR " + first_layer_index + "_" + str(sample_index) + " IS: " + str(rdd_estimate_std_err))
                
        as_orders_rdd_estimate_collection[sample_index] = {}
        as_orders_rdd_estimate_collection[sample_index]["Estimate"] = rdd_estimate_value
        as_orders_rdd_estimate_collection[sample_index]["StdError"] = rdd_estimate_std_err

    print("--- TOTAL RUNTIME OF AS_ORDERS RDD FOR" + first_layer_index + "IS " +  str(time.time() - start_time_00))
        
    return as_orders_rdd_estimate_collection


In [29]:
# 24. AS orders RDD result summaries
def as_orders_rdd_result_summary(as_orders_rdd_estimate_collection, first_layer_index):
    
    print("========== SUMMARIZING AS_ORDERS RDD RESULTS FOR " + first_layer_index + " ==========")
    start_time_00 = time.time()
    
    as_orders_rdd_estimate_value_list = []
    as_orders_rdd_estimate_std_error_list = []

    for result_index, result_sample in as_orders_rdd_estimate_collection.items():

        as_orders_rdd_estimate_value_list += [result_sample["Estimate"]]
        as_orders_rdd_estimate_std_error_list += [result_sample["StdError"]]

    if len(as_orders_rdd_estimate_value_list) == 1:
        final_as_orders_estimate_value = as_orders_rdd_estimate_value_list[0]
        final_as_orders_estimate_std_error = as_orders_rdd_estimate_std_error_list[0]
        final_as_orders_z_score = final_as_orders_estimate_value/final_as_orders_estimate_std_error
        final_as_orders_statistical_significance_p05 = np.abs(final_as_orders_z_score) >= 1.96
        final_as_orders_statistical_significance_p10 = np.abs(final_as_orders_z_score) >= 1.645
        print("(AS_ORDERS) FINAL RDD ESTIMATE FOR " + first_layer_index + " IS: " + str(final_as_orders_estimate_value))
        print("(AS_ORDERS) FINAL RDD ESTIMATE STD ERROR FOR " + first_layer_index + " IS: "+ str(final_as_orders_estimate_std_error))
        print("(AS_ORDERS) FINAL Z SCORE FOR " + first_layer_index + " IS: " + str(final_as_orders_z_score))
        print("(AS_ORDERS) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 5% FOR " + first_layer_index + " IS: " + str(final_as_orders_statistical_significance_p05))
        print("(AS_ORDERS) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 10% FOR " + first_layer_index + " IS: " + str(final_as_orders_statistical_significance_p10))

    else:

        np_as_orders_rdd_estimate_values = np.array(as_orders_rdd_estimate_value_list)
        np_as_orders_rdd_estimate_values_2 = np_as_orders_rdd_estimate_values**2
        np_as_orders_rdd_estimate_std_errors = np.array(as_orders_rdd_estimate_std_error_list)
        np_as_orders_rdd_estimate_variance = np_as_orders_rdd_estimate_std_errors**2

        # assign equal weights to each estimate
        as_orders_np_weights = np.array([1/len(as_orders_rdd_estimate_value_list)] * len(as_orders_rdd_estimate_value_list))

        # final_estimate_value = np.sum(np.array(rdd_estimate_value_list)*np_optimal_weights)
        final_as_orders_estimate_value = np.sum(np_as_orders_rdd_estimate_values * as_orders_np_weights) 
        final_as_orders_estimate_variance =  np.sum(np_as_orders_rdd_estimate_variance * as_orders_np_weights)\
        + np.sum(np_as_orders_rdd_estimate_values_2 * as_orders_np_weights)\
        - np.square(final_as_orders_estimate_value)
        final_as_orders_estimate_std_error = np.sqrt(final_as_orders_estimate_variance)

        final_as_orders_z_score = final_as_orders_estimate_value / final_as_orders_estimate_std_error
        final_as_orders_statistical_significance_p05 = np.abs(final_as_orders_z_score) >= 1.96
        final_as_orders_statistical_significance_p10 = np.abs(final_as_orders_z_score) >= 1.645
        print("(AS_ORDERS) FINAL RDD ESTIMATE FOR: " + str(final_as_orders_estimate_value))
        print("(AS_ORDERS) FINAL RDD ESTIMATE STD ERROR FOR " + str(final_as_orders_estimate_std_error))
        print("(AS_ORDERS) FINAL Z SCORE FOR " + first_layer_index + " IS: " + str(final_as_orders_z_score))
        print("(AS_ORDERS) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 5% FOR " + first_layer_index + " IS: " + str(final_as_orders_statistical_significance_p05))
        print("(AS_ORDERS) FINAL STATISTICAL SIGNIFICANCE CONFIDENCE LEVEL 10% FOR " + first_layer_index + " IS: " + str(final_as_orders_statistical_significance_p10))
        
    finalized_as_orders_rdd_estimates = {}
    finalized_as_orders_rdd_estimates["Estimate"] = final_as_orders_estimate_value
    finalized_as_orders_rdd_estimates["StdError"] = final_as_orders_estimate_std_error
    finalized_as_orders_rdd_estimates["ZScore"] = final_as_orders_z_score
    finalized_as_orders_rdd_estimates["Significance_5_perc"] = final_as_orders_statistical_significance_p05
    finalized_as_orders_rdd_estimates["Significance_10_perc"] = final_as_orders_statistical_significance_p10 
   
    print("--- TOTAL RUNTIME OF AS_ORDERS RDD RESULT SUMMARY FOR" + first_layer_index + "IS " +  str(time.time() - start_time_00))
    
    return finalized_as_orders_rdd_estimates


In [30]:
# 25. Generate pd_valid_as_orders_rdd_estimates
def generate_pd_valid_as_orders_rdd_estimates(finalized_as_orders_rdd_estimates):
    
    k = 0

    for key, value in finalized_as_orders_rdd_estimates.items():

        key_list = key.split("_")
        Product = key_list[0]
        Year = key_list[1]
        Month = key_list[2]

        Estimate = value["Estimate"]
        StdError = value["StdError"]
        ZScore = value["ZScore"]
        Significance_5_perc = value["Significance_5_perc"]
        Significance_10_perc = value["Significance_10_perc"]

        if Significance_10_perc == True:
            if k == 0:
                valid_rdd_estimates_data = [[Product, Year, Month, Estimate, StdError]]
            else:
                valid_rdd_estimates_data = valid_rdd_estimates_data +  [[Product, Year, Month, Estimate, StdError]]

            k = k + 1

    pd_valid_rdd_estimates = pd.DataFrame(valid_rdd_estimates_data, columns = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH", "ESTIMATE", "STD_ERROR"])
    pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"] = pd_valid_rdd_estimates["PRODUCT_PART_NUMBER"].astype(str)
    return pd_valid_rdd_estimates

In [31]:
# 26. Generate time values & product x time values
def generate_as_orders_productxtime_sets(year_list, month_list, product_list, pd_sku_features):
    
    # 3.2.1. time values 
    time_values = [[m1, m2] for m1 in year_list for m2 in month_list]
    pd_time_values = pd.DataFrame(time_values, columns = ["YEAR", "MONTH"])
    pd_time_values["YEAR"] = pd_time_values["YEAR"].astype(str)
    pd_time_values["MONTH"] = pd_time_values["MONTH"].astype(str)

    # 3.2.2. product-time values 
    product_x_time_values = [[m1,m2,m3] for m1 in product_list for m2 in year_list for m3 in month_list]
    pd_product_x_time_values = pd.DataFrame(product_x_time_values, columns = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH"])
    pd_product_x_time_values["PRODUCT_PART_NUMBER"] = pd_product_x_time_values["PRODUCT_PART_NUMBER"].astype(str)
    pd_product_x_time_values["YEAR"] = pd_product_x_time_values["YEAR"].astype(str)
    pd_product_x_time_values["MONTH"] = pd_product_x_time_values["MONTH"].astype(str)
    
    # 3.2.3. Add features
    pd_product_x_time_with_sku_features = pd_product_x_time_values.merge(pd_sku_features, on = ["PRODUCT_PART_NUMBER"], how = "left")
    
    return pd_time_values, pd_product_x_time_with_sku_features

In [32]:
# 27. Remediation: Generate final remediated estimates

def generate_as_orders_remediated_estimates(year_list, month_list, pd_time_values, pd_valid_rdd_estimates_with_sku_features, pd_product_x_time_with_sku_features):

    k=0
    for year in year_list:
        for month in month_list:

            # Get the slice that spans month 0 to the current month 
            pd_time_values_month0_to_current_month = pd_time_values.iloc[0:(k+1),:]
            pd_valid_rdd_estimates_with_sku_features_seed = pd_valid_rdd_estimates_with_sku_features.merge(pd_time_values_month0_to_current_month, on = ["YEAR", "MONTH"])

            # Get the slice that covers the current month (for constructing the complete results for the current month)
            pd_time_values_current_month = pd_time_values.iloc[k:(k+1),:]
            pd_valid_rdd_estimates_with_sku_features_current_month = pd_valid_rdd_estimates_with_sku_features.merge(pd_time_values_current_month, on = ["YEAR", "MONTH"])

            # Get the slice that covers the current month (for constructing the complete results for the current month – serving as left side)
            pd_product_x_time_with_sku_features_current_month =  pd_product_x_time_with_sku_features.merge(pd_time_values_current_month, on = ["YEAR", "MONTH"])

            # Summarize by SKU
            avg_estimate_by_SKU = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["PRODUCT_PART_NUMBER"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_SKU.columns = ["PRODUCT_PART_NUMBER", "SKU_AVG_ESTIMATE"]

            # Summarize by MC1-MC2-MC3 combination
            avg_estimate_by_MC1_MC2_MC3 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1","MC2","MC3"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1_MC2_MC3.columns = ["MC1","MC2","MC3", "MC1_MC2_MC3_AVG_ESTIMATE"]

            # Summarize by MC1-MC2 combination
            avg_estimate_by_MC1_MC2 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1", "MC2"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1_MC2.columns = ["MC1", "MC2", "MC1_MC2_AVG_ESTIMATE"]

            # Summarize by MC1
            avg_estimate_by_MC1 = pd_valid_rdd_estimates_with_sku_features_seed.groupby(["MC1"])["ESTIMATE"].mean().reset_index()
            avg_estimate_by_MC1.columns = ["MC1", "MC1_AVG_ESTIMATE"]

            # Overall estimate
            avg_estimate_overall = np.mean(pd_valid_rdd_estimates_with_sku_features_seed["ESTIMATE"])

            # Now we merge the average results with current month results to complete remediation 
            matching_vars = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH", "MC1", "MC2", "MC3"]
            pd_product_x_time_with_sku_features_current_month_remediated = pd_product_x_time_with_sku_features_current_month.merge(pd_valid_rdd_estimates_with_sku_features_current_month, on = matching_vars, how = "left")\
            .merge(avg_estimate_by_SKU, on = ["PRODUCT_PART_NUMBER"], how = "left")\
            .merge(avg_estimate_by_MC1_MC2_MC3, on = ["MC1", "MC2", "MC3"], how = "left")\
            .merge(avg_estimate_by_MC1_MC2, on = ["MC1", "MC2"], how = "left")\
            .merge(avg_estimate_by_MC1, on = ["MC1"], how = "left")
            pd_product_x_time_with_sku_features_current_month_remediated["AVERAGE_ESTIMATE_OVERALL"] = avg_estimate_overall

            # Fill in missing values in the order of organic estimate, SKU average, MC1-MC2-MC3 average, MC1-MC2 average, M1 average, overall average 
            pd_product_x_time_with_sku_features_current_month_remediated["REMEDIATED_ESTIMATE"] = np.where(~pd_product_x_time_with_sku_features_current_month_remediated["ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["ESTIMATE"],
                                                                                                          np.where(~pd_product_x_time_with_sku_features_current_month_remediated["SKU_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["SKU_AVG_ESTIMATE"],
                                                                                                                  np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_MC3_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_MC3_AVG_ESTIMATE"],
                                                                                                                          np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_MC2_AVG_ESTIMATE"], 
                                                                                                                                  np.where(~pd_product_x_time_with_sku_features_current_month_remediated["MC1_AVG_ESTIMATE"].isnull(), pd_product_x_time_with_sku_features_current_month_remediated["MC1_AVG_ESTIMATE"], pd_product_x_time_with_sku_features_current_month_remediated["AVERAGE_ESTIMATE_OVERALL"])))))


            # Construct the complete estimates from month 0 to current month
            if k == 0:
                pd_remediated_estimates = pd_product_x_time_with_sku_features_current_month_remediated		
            else:
                pd_remediated_estimates = pd_remediated_estimates.append(pd_product_x_time_with_sku_features_current_month_remediated)

    
            k=k+1
        
    pd_remediated_estimates = pd_remediated_estimates.reset_index()    
            
    return pd_remediated_estimates

# 2. Algorithm Pipeline

In [ ]:
# 1. CP data
cp_data_path = "s3a://prd-use1-datascientists-sc-fp-data/prd/aero/AERO_ORDER_PLACED_WITH_CP_DATA/"
df_cp = read_cp_data(cp_data_path)

start_time = time.time()
print("--- Complete CP data points: " + str(df_cp.count()) + "---")
print("--- %s seconds ---" % (time.time() - start_time))

# Part 1. Clickstream RDD
# 1. Complete clickstream data
clickstream_data_path = "s3a://prd-use1-datascientists-sc-fp-data/prd/aero/AERO_CLICK_STREAM_SUBSAMPLE/"
df_clickstream_data = read_clickstream_data(clickstream_data_path)
start_time = time.time()
print("--- Complete clickstream data points: " + str(df_clickstream_data.count()) + "---")
print("--- %s seconds ---" % (time.time() - start_time))

# 2. Complete rdd eligible sku-year-month combinations 
pd_clickstream_sku_year_month_eligible_complete = pd.read_csv("instock_value_clickstream_candidate_sku_months.csv")\
                                                    .sort_values(["YEAR","MONTH","PRODUCT_PART_NUMBER"])
pd_clickstream_sku_year_month_eligible_complete["YEAR"] = pd_clickstream_sku_year_month_eligible_complete["YEAR"].astype(str)
pd_clickstream_sku_year_month_eligible_complete["MONTH"] = pd_clickstream_sku_year_month_eligible_complete["MONTH"].astype(str)
pd_clickstream_sku_year_month_eligible_complete["PRODUCT_PART_NUMBER"] = pd_clickstream_sku_year_month_eligible_complete["PRODUCT_PART_NUMBER"].astype(str)

# 
pd_sku_features = pd.read_csv("sku_features.csv")

# 3. The combinations requested
# 3.1. product, year, months requested
pd_sku_needed = pd.read_csv("Aristotle_AB_Experiment_Item_Selection.csv")
pd_sku_needed.columns = [x.upper() for x in pd_sku_needed.columns.to_list()]
clickstream_year_list = [str(x) for x in [2020, 2021, 2022]]
clickstream_month_list = [str(x) for x in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
clickstream_product_list = pd_sku_needed["PRODUCT_PART_NUMBER"].to_list()
# 3.2. product x year x month complete set requested
pd_clickstream_time_values, pd_clickstream_product_x_time_with_sku_features = generate_clickstream_productxtime_sets(clickstream_year_list, clickstream_month_list, clickstream_product_list, pd_sku_features)
# 3.3. product x year x month requested that is possibly rdd eligible 
pd_clickstream_sku_year_month_eligible_requested = pd_clickstream_sku_year_month_eligible_complete.merge(pd_clickstream_product_x_time_with_sku_features, on = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH"], how = "inner")

# 4. 
#
s = 1
stopper = 27
clickstream_rdd_customers = {}
clickstream_rdd_sample_collection = {}
for year in clickstream_year_list:
    for month in clickstream_month_list:
        
        # latest result: Feb, 22
        if s > stopper:
            break
        
        else:
            
            # Get the subset of the clickstream data
            zero_layer_index = str(year) + "_" + str(month)


            # Filter out the month's data
            df_clickstream_data_slice = df_clickstream_data.where((F.col("SESSION_YEAR") == year) \
                                                    & (F.col("SESSION_MONTH") == month))

            pd_clickstream_sku_year_month_eligible_requested_monthly_slice = pd_clickstream_sku_year_month_eligible_requested[ (pd_clickstream_sku_year_month_eligible_requested["YEAR"] == year) \
                                                                        & (pd_clickstream_sku_year_month_eligible_requested["MONTH"] == month) ].reset_index()

            nNum = len(pd_clickstream_sku_year_month_eligible_requested_monthly_slice)
            print("(CLICKSTREAM) # OF REQUESTED RDD ELIGIBLE EXAMPLES FOR YEAR " + str(year) + " AND MONTH " + str(month) + " IS " + str(nNum))

            #
            blockSize = 25
            m = 0 
            block_index = 0
            clickstream_rdd_customers[zero_layer_index] = {}

            while m <= nNum:

                clickstream_rdd_customers_list = []

                pd_clickstream_sku_year_month_eligible_requested_monthly_slice_block = pd_clickstream_sku_year_month_eligible_requested_monthly_slice.iloc[m:min(nNum,m+blockSize),:]
                product_list = pd_clickstream_sku_year_month_eligible_requested_monthly_slice_block["PRODUCT_PART_NUMBER"].to_list()
                df_clickstream_data_slice_block = df_clickstream_data_slice.filter(F.col("PRODUCT_PART_NUMBER").isin(product_list))

                start_time = time.time()
                print("CLICKSTREAM OF " + zero_layer_index + " BLOCK HAS " + str(df_clickstream_data_slice_block.count()))
                print("--- %s seconds --- " + str(time.time() - start_time))

                # Cache the data for faster computation
                df_clickstream_data_slice_block.persist()
              
                # Get RDD relevant customers for this block of requested examples 
                start_time_0 = time.time()
                start_time_00 = time.time()

                pd_clickstream_data_sku_year_month_dict = {}
                pd_clickstream_cutoff_point_detection_dict = {}
                pd_clickstream_cutoff_points_dict = {}

                for ix0, row0 in pd_clickstream_sku_year_month_eligible_requested_monthly_slice.iterrows():

                    sku = row0["PRODUCT_PART_NUMBER"]
                    first_layer_index = str(sku) + "_" + str(year) + "_" + str(month)
                    df_clickstream_data_sku_year_month, pd_clickstream_data_sku_year_month = clickstream_slice_pandas(df_clickstream_data_slice_block, sku, year, month)
                    pd_clickstream_data_sku_year_month_dict[first_layer_index] = pd_clickstream_data_sku_year_month

                    pd_clickstream_cutoff_point_detection, pd_clickstream_cutoff_points = find_clickstream_cutoff_dates(pd_clickstream_data_sku_year_month)
                    pd_clickstream_cutoff_point_detection_dict[first_layer_index] = pd_clickstream_cutoff_point_detection
                    pd_clickstream_cutoff_points_dict[first_layer_index] = pd_clickstream_cutoff_points                

                    sufficiency_ratio = 0.5
                    initial_window_length = 7
                    clickstream_rdd_customers_temp = find_clickstream_rdd_relevant_customers(pd_clickstream_cutoff_points,
                                                                                                  pd_clickstream_cutoff_point_detection,
                                                                                                  pd_clickstream_data_sku_year_month,
                                                                                                  sufficiency_ratio, initial_window_length,
                                                                                                  first_layer_index)
                    clickstream_rdd_customers_list = [*set(clickstream_rdd_customers_list+clickstream_rdd_customers_temp)]


                    if (ix0 > 0) & (ix0 % 10 == 0):
                        print("(CLICKSTREAM) " + str(ix0/nNum*100) + "%" + " COMPLETED FOR " + zero_layer_index)
                        print("--- %s seconds --- " + str(time.time() - start_time_00))
                        start_time_00 = time.time()


                print("(CLICKSTREAM) THE NUM. OF RDD RELEVANT CUSTOMERS IN " + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS " + str(len(clickstream_rdd_customers_list)))
                print("--- %s seconds --- " + str(time.time() - start_time_0))
                clickstream_rdd_customers[zero_layer_index]["BLOCK" + "_" + str(block_index)] = clickstream_rdd_customers_list

                # save the results (of the month)
                # save_file_name = "clickstream_rdd_customers_" + zero_layer_index + ".pkl"

                # with open(save_file_name, "wb") as fp:
                #    pickle.dump(clickstream_rdd_customers, fp)
                #    print("RDD CUSTOMERS OF" + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS SAVED.")



                # Get the corresponding relevant portion of CP data
                df_cp_block = df_cp.filter(F.col("CUSTOMER_ID").isin(clickstream_rdd_customers_list))
                
                # Cache the data for faster computation
                df_cp_block.persist()

                start_time = time.time()
                print("(CLICKSTREAM) THE NUMBER OF RELEVANT CP DATA POINTS OF" + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS " + str(df_cp_block.count()))
                print("--- %s seconds ---" + str(time.time() - start_time))
                
                i = 0
                start_time = time.time()
                start_time_00 = time.time()
                for ix0, row0 in pd_clickstream_sku_year_month_eligible_requested_monthly_slice.iterrows():

                    # timer
                    start_time_initial = time.time()

                    # SKU, Year, Month
                    sku = row0["PRODUCT_PART_NUMBER"]
                    first_layer_index = str(sku) + "_" + str(year) + "_" + str(month)

                    print("--- WORKING ON INDEX: " + str(i) + " EXAMPLE NO. " + first_layer_index + " ---")

                    # Part 1. Clickstream Portion of Instock Value  

                    # 1. Clickstream slice
                    pd_clickstream_data_sku_year_month = pd_clickstream_data_sku_year_month_dict[first_layer_index]
                    print("CLICKSTREAM COUNTS FOR " + first_layer_index + " IS " + str(len(pd_clickstream_data_sku_year_month)))

                    # 3. Clickstream cutoff Date Data
                    pd_clickstream_cutoff_point_detection = pd_clickstream_cutoff_point_detection_dict[first_layer_index]
                    pd_clickstream_cutoff_points = pd_clickstream_cutoff_points_dict[first_layer_index]     

                    print("(CLICKSTREAM) CUTOFF POINTS OF " + first_layer_index + " IS ")
                    print(pd_clickstream_cutoff_points)

                    # 4. RDD regression sample construction
                    sufficiency_ratio = 0.5 
                    initial_window_length = 7
                    clickstream_rdd_sample_collection[first_layer_index] = construct_clickstream_rdd_samples(pd_clickstream_cutoff_points, pd_clickstream_cutoff_point_detection, pd_clickstream_data_sku_year_month, sufficiency_ratio, initial_window_length, first_layer_index, df_cp_block)

                    # with open('clickstream_rdd_sample_collection_part2.pkl', 'wb') as fp:
                    #    pickle.dump(clickstream_rdd_sample_collection, fp)
                    #    print('dictionary saved successfully to file')

                    # with open('clickstream_rdd_sample_collection_part2.pkl', 'rb') as fp:
                    #    clickstream_rdd_sample_collection = pickle.load(fp)
                    #    print('Reading the RDD samples')

                    # 5. Regression and Result Summaries
                    # CP_V3: CP of ALL products OTHER THAN the one of interest
                    cp_variable = "CP_V3"

                    # Polynomial order of running variable
                    K = 1
                    # Three types: SEPARATE_UNWEIGHTED (Run RDD for each sample with OLS), SEPARATE_WEIGHTED (Run RDD for each sample with WLS), JOINT_WEIGHTED (Run RDD on a combined sample with WLS)
                    regression_type = "JOINT_WEIGHTED"
                    # assign higher weights to data points within 24 hours of the cutoff point
                    h = 24
                    a1 = 1
                    a2 = 2
                    bootstrapping_method = True
                    boostramp_sample_size = 2500
                    if len(clickstream_rdd_sample_collection[first_layer_index]) == 0:
                        continue
                    else:
                        clickstream_rdd_estimate_collection[first_layer_index] = clickstream_rdd_modeling(clickstream_rdd_sample_collection[first_layer_index], first_layer_index, cp_variable, K, regression_type, h, a1, a2, bootstrapping_method, boostramp_sample_size)
                        finalized_clickstream_rdd_estimates[first_layer_index] = clickstream_rdd_result_summary(clickstream_rdd_estimate_collection[first_layer_index], first_layer_index)

                    with open("finalized_clickstream_rdd_estimates.pkl", "wb") as fp:
                        pickle.dump(finalized_clickstream_rdd_estimates, fp)
                        print("(CLICKSTREAM) RDD ESTIMATES dictionary saved successfully to file")    
                    
                    print("--- %s seconds ---" + str(time.time() - start_time_00))
                    start_time_00 = time.time()
                    i = i + 1
                
                print("--- %s seconds ---" + str(time.time() - start_time))
                m = m + blockSize
                block_index = block_index + 1
                df_cp_block.unpersist()

# Part 2. As_orders RDD
# 1. Complete as_orders data
as_orders_data_path = "s3a://prd-use1-datascientists-sc-fp-data/prd/aero/AERO_AS_ORDERS_SUBSAMPLE/"
df_as_orders_data = read_as_orders_data(as_orders_data_path)
start_time = time.time()
print("--- Complete as_orders data points: " + str(df_as_orders_data.count()) + "---")
print("--- %s seconds ---" % (time.time() - start_time))

# 2. Complete rdd eligible sku-year-month combinations 
pd_as_orders_sku_year_month_eligible_complete = pd.read_csv("instock_value_as_orders_candidate_sku_months.csv")\
                                                    .sort_values(["YEAR","MONTH","PRODUCT_PART_NUMBER"])
pd_as_orders_sku_year_month_eligible_complete["YEAR"] = pd_as_orders_sku_year_month_eligible_complete["YEAR"].astype(str)
pd_as_orders_sku_year_month_eligible_complete["MONTH"] = pd_as_orders_sku_year_month_eligible_complete["MONTH"].astype(str)
pd_as_orders_sku_year_month_eligible_complete["PRODUCT_PART_NUMBER"] = pd_as_orders_sku_year_month_eligible_complete["PRODUCT_PART_NUMBER"].astype(str)

# 
pd_sku_features = pd.read_csv("sku_features.csv")

# 3. The combinations requested
# 3.1. product, year, months requested
pd_sku_needed = pd.read_csv("Aristotle_AB_Experiment_Item_Selection.csv")
pd_sku_needed.columns = [x.upper() for x in pd_sku_needed.columns.to_list()]
as_orders_year_list = [str(x) for x in [2020, 2021, 2022]]
as_orders_month_list = [str(x) for x in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
as_orders_product_list = pd_sku_needed["PRODUCT_PART_NUMBER"].to_list()
# 3.2. product x year x month complete set requested
pd_as_orders_time_values, pd_as_orders_product_x_time_with_sku_features = generate_as_orders_productxtime_sets(as_orders_year_list, as_orders_month_list, as_orders_product_list, pd_sku_features)
# 3.3. product x year x month requested that is possibly rdd eligible 
pd_as_orders_sku_year_month_eligible_requested = pd_as_orders_sku_year_month_eligible_complete.merge(pd_as_orders_product_x_time_with_sku_features, on = ["PRODUCT_PART_NUMBER", "YEAR", "MONTH"], how = "inner")

# 4. 
#
s = 1
stopper = 27
as_orders_rdd_customers = {}
as_orders_rdd_sample_collection = {}
for year in as_orders_year_list:
    for month in as_orders_month_list:
        
        # latest result: Feb, 22
        if s > stopper:
            break
        
        else:
            
            # Get the subset of the as_orders data
            zero_layer_index = str(year) + "_" + str(month)

            # Filter out the month's data
            df_as_orders_data_slice = df_as_orders_data.where((F.col("ORDER_PLACED_YEAR") == year) \
                                                    & (F.col("ORDER_PLACED_MONTH") == month))

            pd_as_orders_sku_year_month_eligible_requested_monthly_slice = pd_as_orders_sku_year_month_eligible_requested[ (pd_as_orders_sku_year_month_eligible_requested["YEAR"] == year) \
                                                                        & (pd_as_orders_sku_year_month_eligible_requested["MONTH"] == month) ].reset_index()

            nNum = len(pd_as_orders_sku_year_month_eligible_requested_monthly_slice)
            print("(AS_ORDERS) # OF REQUESTED RDD ELIGIBLE EXAMPLES FOR YEAR " + str(year) + " AND MONTH " + str(month) + " IS " + str(nNum))

            #
            blockSize = 25
            m = 0 
            block_index = 0
            as_orders_rdd_customers[zero_layer_index] = {}

            while m <= nNum:

                as_orders_rdd_customers_list = []

                pd_as_orders_sku_year_month_eligible_requested_monthly_slice_block = pd_as_orders_sku_year_month_eligible_requested_monthly_slice.iloc[m:min(nNum,m+blockSize),:]
                product_list = pd_as_orders_sku_year_month_eligible_requested_monthly_slice_block["PRODUCT_PART_NUMBER"].to_list()
                df_as_orders_data_slice_block = df_as_orders_data_slice.filter(F.col("PRODUCT_PART_NUMBER").isin(product_list))

                start_time = time.time()
                print("AS_ORDERS OF " + zero_layer_index + " BLOCK HAS " + str(df_as_orders_data_slice_block.count()))
                print("--- %s seconds --- " + str(time.time() - start_time))

                # Cache the data for faster computation
                df_as_orders_data_slice_block.persist()
              
                # Get RDD relevant customers for this block of requested examples 
                start_time_0 = time.time()
                start_time_00 = time.time()

                pd_as_orders_data_sku_year_month_dict = {}
                pd_as_orders_cutoff_point_detection_dict = {}
                pd_as_orders_cutoff_points_dict = {}

                for ix0, row0 in pd_as_orders_sku_year_month_eligible_requested_monthly_slice.iterrows():

                    sku = row0["PRODUCT_PART_NUMBER"]
                    first_layer_index = str(sku) + "_" + str(year) + "_" + str(month)
                    df_as_orders_data_sku_year_month, pd_as_orders_data_sku_year_month = as_orders_slice_pandas(df_as_orders_data_slice_block, sku, year, month)
                    pd_as_orders_data_sku_year_month_dict[first_layer_index] = pd_as_orders_data_sku_year_month

                    pd_as_orders_cutoff_point_detection, pd_as_orders_cutoff_points = find_as_orders_cutoff_dates(pd_as_orders_data_sku_year_month)
                    pd_as_orders_cutoff_point_detection_dict[first_layer_index] = pd_as_orders_cutoff_point_detection
                    pd_as_orders_cutoff_points_dict[first_layer_index] = pd_as_orders_cutoff_points                

                    sufficiency_ratio = 0.5
                    initial_window_length = 7
                    as_orders_rdd_customers_temp = find_as_orders_rdd_relevant_customers(pd_as_orders_cutoff_points,
                                                                                                  pd_as_orders_cutoff_point_detection,
                                                                                                  pd_as_orders_data_sku_year_month,
                                                                                                  sufficiency_ratio, initial_window_length,
                                                                                                  first_layer_index)
                    as_orders_rdd_customers_list = [*set(as_orders_rdd_customers_list+as_orders_rdd_customers_temp)]


                    if (ix0 > 0) & (ix0 % 10 == 0):
                        print("(AS_ORDERS) " + str(ix0/nNum*100) + "%" + " COMPLETED FOR " + zero_layer_index)
                        print("--- %s seconds --- " + str(time.time() - start_time_00))
                        start_time_00 = time.time()


                print("(AS_ORDERS) THE NUM. OF RDD RELEVANT CUSTOMERS IN " + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS " + str(len(as_orders_rdd_customers_list)))
                print("--- %s seconds --- " + str(time.time() - start_time_0))
                as_orders_rdd_customers[zero_layer_index]["BLOCK" + "_" + str(block_index)] = as_orders_rdd_customers_list

                # save the results (of the month)
                # save_file_name = "as_orders_rdd_customers_" + zero_layer_index + ".pkl"

                # with open(save_file_name, "wb") as fp:
                #    pickle.dump(as_orders_rdd_customers, fp)
                #    print("RDD CUSTOMERS OF" + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS SAVED.")

                # Get the corresponding relevant portion of CP data
                df_cp_block = df_cp.filter(F.col("CUSTOMER_ID").isin(as_orders_rdd_customers_list))
                
                # Cache the data for faster computation
                df_cp_block.persist()

                start_time = time.time()
                print("(AS_ORDERS) THE NUMBER OF RELEVANT CP DATA POINTS OF" + " BLOCK " + str(block_index) + " OF " + zero_layer_index + " IS " + str(df_cp_block.count()))
                print("--- %s seconds ---" + str(time.time() - start_time))
                
                i = 0
                start_time = time.time()
                start_time_00 = time.time()
                for ix0, row0 in pd_as_orders_sku_year_month_eligible_requested_monthly_slice.iterrows():

                    # timer
                    start_time_initial = time.time()

                    # SKU, Year, Month
                    sku = row0["PRODUCT_PART_NUMBER"]
                    first_layer_index = str(sku) + "_" + str(year) + "_" + str(month)

                    print("--- WORKING ON INDEX: " + str(i) + " EXAMPLE NO. " + first_layer_index + " ---")

                    # Part 1. As_orders Portion of Instock Value  

                    # 1. As_orders slice
                    pd_as_orders_data_sku_year_month = pd_as_orders_data_sku_year_month_dict[first_layer_index]
                    print("AS_ORDERS COUNTS FOR " + first_layer_index + " IS " + str(len(pd_as_orders_data_sku_year_month)))

                    # 3. As_orders cutoff Date Data
                    pd_as_orders_cutoff_point_detection = pd_as_orders_cutoff_point_detection_dict[first_layer_index]
                    pd_as_orders_cutoff_points = pd_as_orders_cutoff_points_dict[first_layer_index]     

                    print("(AS_ORDERS) CUTOFF POINTS OF " + first_layer_index + " IS ")
                    print(pd_as_orders_cutoff_points)

                    # 4. RDD regression sample construction
                    sufficiency_ratio = 0.5 
                    initial_window_length = 7
                    as_orders_rdd_sample_collection[first_layer_index] = construct_as_orders_rdd_samples(pd_as_orders_cutoff_points, pd_as_orders_cutoff_point_detection, pd_as_orders_data_sku_year_month, sufficiency_ratio, initial_window_length, first_layer_index, df_cp_block)

                    # with open('as_orders_rdd_sample_collection_part2.pkl', 'wb') as fp:
                    #    pickle.dump(as_orders_rdd_sample_collection, fp)
                    #    print('dictionary saved successfully to file')

                    # with open('as_orders_rdd_sample_collection_part2.pkl', 'rb') as fp:
                    #    as_orders_rdd_sample_collection = pickle.load(fp)
                    #    print('Reading the RDD samples')

                    # 5. Regression and Result Summaries
                    # CP_V3: CP of ALL products OTHER THAN the one of interest
                    cp_variable = "CP_V3"

                    # Polynomial order of running variable
                    K = 1
                    # Three types: SEPARATE_UNWEIGHTED (Run RDD for each sample with OLS), SEPARATE_WEIGHTED (Run RDD for each sample with WLS), JOINT_WEIGHTED (Run RDD on a combined sample with WLS)
                    regression_type = "JOINT_WEIGHTED"
                    # assign higher weights to data points within 24 hours of the cutoff point
                    h = 24
                    a1 = 1
                    a2 = 2
                    bootstrapping_method = True
                    boostramp_sample_size = 2500
                    if len(as_orders_rdd_sample_collection[first_layer_index]) == 0:
                        continue
                    else:
                        as_orders_rdd_estimate_collection[first_layer_index] = as_orders_rdd_modeling(as_orders_rdd_sample_collection[first_layer_index], first_layer_index, cp_variable, K, regression_type, h, a1, a2, bootstrapping_method, boostramp_sample_size)
                        finalized_as_orders_rdd_estimates[first_layer_index] = as_orders_rdd_result_summary(as_orders_rdd_estimate_collection[first_layer_index], first_layer_index)

                    with open("finalized_as_orders_rdd_estimates.pkl", "wb") as fp:
                        pickle.dump(finalized_as_orders_rdd_estimates, fp)
                        print("(AS_ORDERS) RDD ESTIMATES dictionary saved successfully to file")    
                    
                    print("--- %s seconds ---" + str(time.time() - start_time_00))
                    start_time_00 = time.time()
                    i = i + 1
                
                print("--- %s seconds ---" + str(time.time() - start_time))
                m = m + blockSize
                block_index = block_index + 1

# 3. Remediation
# 3.1 Clickstream
# Step 1. Read from the RDD estimate dictionary into Pandas dataframe
with open("finalized_clickstream_rdd_estimates.pkl", "rb") as fp:
    finalized_clickstream_rdd_estimates = pickle.load(fp)
    print("(CLICKSTREAM) RDD ESTIMATES SUCCESSFULLY READ") 
pd_clickstream_valid_rdd_estimates = generate_pd_valid_clickstream_rdd_estimates(finalized_clickstream_rdd_estimates)

# Step 2. Read in the product features data and add features to valid rdd estimate data
pd_sku_features = pd.read_csv("sku_features.csv")
pd_sku_features.columns = ["PRODUCT_PART_NUMBER", "PRODUCT_NAME", "MC1", "MC2", "MC3"]
pd_clickstream_valid_rdd_estimates_with_sku_features = pd_clickstream_valid_rdd_estimates.merge(pd_sku_features, on = ["PRODUCT_PART_NUMBER"], how = "left")

# Step 3. Remediation algorithm
# 3.1. Create the complete list of values
pd_sku_needed = pd.read_csv("Aristotle_AB_Experiment_Item_Selection.csv")
pd_sku_needed.columns = [x.upper() for x in pd_sku_needed.columns.to_list()]

clickstream_year_list = [2020, 2021, 2022]
clickstream_month_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
clickstream_product_list = pd_sku_needed["PRODUCT_PART_NUMBER"].to_list()

# 3.2. Create product x time complete set
pd_clickstream_time_values, pd_clickstream_product_x_time_with_sku_features = generate_clickstream_productxtime_sets(clickstream_year_list, clickstream_month_list, clickstream_product_list, pd_sku_features)

# Step 4. Remediation
pd_clickstream_remediated_estimates = generate_as_orders_remediated_estimates(clickstream_year_list, clickstream_month_list, pd_clickstream_time_values, pd_clickstream_valid_rdd_estimates_with_sku_features, pd_clickstream_product_x_time_with_sku_features)


# 3.2. As_orders
# Step 1. Read from the RDD estimate dictionary into Pandas dataframe
pd_as_orders_valid_rdd_estimates = generate_pd_valid_as_orders_rdd_estimates(finalized_as_orders_rdd_estimates)

# Step 2. Read in the product features data and add features to valid rdd estimate data
# pd_sku_features = pd.read_csv()
pd_as_orders_valid_rdd_estimates_with_sku_features = pd_as_orders_valid_rdd_estimates.merge(pd_sku_features, on = ["PRODUCT_PART_NUMBER"], how = "left")


# Step 3. Remediation algorithm
# 3.1. Create the complete list of values
as_orders_year_list = clickstream_year_list
as_orders_month_list = clickstream_month_list
as_orders_product_list = clickstream_product_list

# 3.2. Create product x time complete set
pd_as_orders_time_values, pd_as_orders_product_x_time_with_sku_features = generate_as_orders_productxtime_sets(as_orders_year_list, as_orders_month_list, as_orders_product_list, pd_sku_features)

# Step 4. Remediation
pd_as_orders_remediated_estimates = generate_as_orders_remediated_estimates(as_orders_year_list, as_orders_month_list, pd_as_orders_time_values, pd_as_orders_valid_rdd_estimates_with_sku_features, pd_as_orders_product_x_time_with_sku_features)
                
        
# Part 4. Two key ingredients for clickstream portion of instock value: non-AS demand share & clickstream counts per unit of non-AS demand
# 4.1. Clickstream Counts
is_clickstream_counts, oos_clickstream_counts, total_clickstream_counts = clickstream_counts_by_instock_status(pd_clickstream_data_slice)

print("IS CLICKSTREAM COUNTS FOR " + first_layer_index + " IS " + str(is_clickstream_counts))
print("OOS CLICKSTREAM COUNTS FOR " + first_layer_index + " IS " + str(oos_clickstream_counts))
print("TOTAL CLICKSTREAM COUNTS FOR " + first_layer_index + " IS " + str(total_clickstream_counts))

# 4.2. CP (orders) data
df_cp_sku_month = df_cp.filter( (F.col("PRODUCT_PART_NUMBER_IN_CP") == str(product_part_number_for_sample)) & \
                        (F.col("ORDER_PLACED_YEAR_CP") == year_value_for_sample) & \
                        (F.col("ORDER_PLACED_MONTH_CP") == month_value_for_sample))

df_cp_sku_month.persist()

# 4.3. Units sold by as status
start_time = time.time()
non_as_units_sold, as_units_sold, total_units_sold = units_sold_by_as_status(df_cp_sku_month)
print("NON-AS SOLD UNITS FOR " + first_layer_index + " IS " + str(non_as_units_sold))    
print("AS SOLD UNITS FOR " + first_layer_index + " IS " + str(as_units_sold))  
print("TOTAL SOLD UNITS FOR " + first_layer_index + " IS " + str(total_units_sold))  
print("--- %s seconds ---" % (time.time() - start_time))

# 4.4. AS & non-AS demand shares
non_as_share = non_as_units_sold/(non_as_units_sold+as_units_sold)
as_share =  1 - non_as_share

print("NON-AS DEMAND SHARE FOR " + first_layer_index + " IS " + str(non_as_share))
print("AS DEMAND SHARE FOR " + first_layer_index + " IS " + str(as_share))

# 4.5. Coefficients: clickstream counts per unit of non-AS demand
clickstream_coef = is_clickstream_counts / non_as_units_sold

print("CLICKSTREAM-NON-AS-DEMAND COEFFICIENT FOR " + first_layer_index + " IS " + str(clickstream_coef))

# 4.6. save the two key elements: non-AS demand share & clickstream coefficient
non_as_demand_share_collection[first_layer_index] = non_as_share
clickstream_non_as_demand_coefficient_collection[first_layer_index] = clickstream_coef

    
